In [1]:
# 1) Install dependencies
!pip install --quiet git+https://github.com/VarunGumma/IndicTransToolkit.git --use-pep517
!pip install --quiet transformers sentencepiece accelerate bitsandbytes datasets peft


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 20.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 23.1 MB/s eta 0:00:00:00:0100:01


In [2]:
import os
import torch
import warnings
from datasets import Dataset, DatasetDict
from IndicTransToolkit.processor import IndicProcessor
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
)
from peft import get_peft_model, LoraConfig

# 2) Setup
warnings.filterwarnings("ignore")
os.environ["WANDB_DISABLED"] = "true"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


Using device: cuda


In [3]:
model_name = "prajdabre/rotary-indictrans2-en-indic-dist-200M"

# 3) Initialize processor, tokenizer, base model
ip = IndicProcessor(inference=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    trust_remote_code=True
)
if device=="cuda":
    base_model = base_model.to(device)


tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenization_indictrans.py:   0%|          | 0.00/8.04k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/prajdabre/rotary-indictrans2-en-indic-dist-200M:
- tokenization_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


dict.SRC.json:   0%|          | 0.00/645k [00:00<?, ?B/s]

dict.TGT.json:   0%|          | 0.00/3.39M [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/prajdabre/rotary-indictrans2-en-indic-dist-200M:
- configuration_rotary_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_rotary_indictrans.py:   0%|          | 0.00/68.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/prajdabre/rotary-indictrans2-en-indic-dist-200M:
- modeling_rotary_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/847M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

In [4]:
# 4) Apply LoRA adapters
target_modules = [n.split(".")[-1] 
                  for n, m in base_model.named_modules() 
                  if "q_proj" in n or "v_proj" in n]
lora_cfg = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=target_modules,
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
)
model = get_peft_model(base_model, lora_cfg)
model.print_trainable_parameters()


trainable params: 884,736 || all params: 212,661,248 || trainable%: 0.4160


In [5]:
colloquial_dataset = [
    { "en": "What's up?", "hi": "क्या हाल है?" },
    { "en": "Chill out, man.", "hi": "शांत हो जाओ, यार।" },
    { "en": "That's so cool!", "hi": "यह तो बहुत बढ़िया है!" },
    {
        "en": "I'm feeling a bit under the weather.",
        "hi": "मेरी तबीयत थोड़ी नासाज है।",
    },
    { "en": "Let's grab a bite.", "hi": "चलो कुछ खाते हैं।" },
    { "en": "Hang in there.", "hi": "लगे रहो।" },
    { "en": "No worries.", "hi": "कोई बात नहीं।" },
    { "en": "It's raining cats and dogs.", "hi": "बहुत तेज बारिश हो रही है।" },
    { "en": "He's a couch potato.", "hi": "वह तो बस घर में पड़ा रहता है।" },
    { "en": "Break a leg!", "hi": "शुभकामनाएं!" },
    { "en": "That's the last straw!", "hi": "अब तो हद हो गई!" },
    { "en": "Spill the beans.", "hi": "सच बताओ।" },
    { "en": "Bite the bullet.", "hi": "कष्ट सह लो।" },
    { "en": "Get your act together.", "hi": "अपने काम पर ध्यान दो।" },
    { "en": "Hit the road.", "hi": "चलो यहाँ से निकलें।" },
    { "en": "Once in a blue moon.", "hi": "कभी कभार।" },
    { "en": "Piece of cake.", "hi": "बहुत आसान।" },
    { "en": "Take it easy.", "hi": "आराम से करो।" },
    { "en": "See eye to eye.", "hi": "एकमत होना।" },
    { "en": "Through thick and thin.", "hi": "अच्छे और बुरे समय में।" },
    { "en": "He's a smart cookie.", "hi": "वह बहुत होशियार है।" },
    { "en": "I'm just pulling your leg.", "hi": "मैं तो बस मजाक कर रहा हूँ।" },
    { "en": "Let's call it a day.", "hi": "चलो आज के लिए बस करते हैं।" },
    { "en": "He's got a chip on his shoulder.", "hi": "वह हमेशा नाराज़ रहता है।" },
    { "en": "I'm on cloud nine.", "hi": "मैं बहुत खुश हूँ।" },
    { "en": "Keep your chin up.", "hi": "हिम्मत मत हारो।" },
    { "en": "Out of the blue.", "hi": "अचानक से।" },
    { "en": "Rule of thumb.", "hi": "अनुभव के आधार पर नियम।" },
    { "en": "That's a tough nut to crack.", "hi": "यह एक मुश्किल समस्या है।" },
    {
        "en": "Time flies when you're having fun.",
        "hi": "मज़े करते हुए समय का पता नहीं चलता।",
    },
    { "en": "We're in the same boat.", "hi": "हमारी स्थिति एक जैसी है।" },
    {
        "en": "You can't judge a book by its cover.",
        "hi": "किसी को बाहरी रूप से मत आंकना।",
    },
    { "en": "He's gone bananas!", "hi": "वह पागल हो गया है!" },
    { "en": "I'm feeling blue.", "hi": "मैं उदास महसूस कर रहा हूँ।" },
    { "en": "Let's paint the town red.", "hi": "चलो खूब मस्ती करते हैं।" },
    {
        "en": "That's the best thing since sliced bread.",
        "hi": "यह तो कमाल की चीज है।",
    },
    { "en": "To kill two birds with one stone.", "hi": "एक तीर से दो शिकार करना।" },
    {
        "en": "Don't count your chickens before they hatch.",
        "hi": "सपने सच होने से पहले उन पर भरोसा मत करो।",
    },
    {
        "en": "Every cloud has a silver lining.",
        "hi": "हर मुश्किल में कुछ अच्छा छिपा होता है।",
    },
    {
        "en": "The early bird catches the worm.",
        "hi": "जो जल्दी उठता है, वह पाता है।",
    },
    { "en": "Practice makes perfect.", "hi": "अभ्यास से सब संभव है।" },
    { "en": "Where there's a will, there's a way.", "hi": "जहाँ चाह, वहाँ राह।" },
    { "en": "A penny saved is a penny earned.", "hi": "बचत ही कमाई है।" },
    {
        "en": "Don't put all your eggs in one basket.",
        "hi": "अपना सब कुछ एक ही जगह मत लगाओ।",
    },
    {
        "en": "The grass is always greener on the other side.",
        "hi": "दूर के ढोल सुहावने होते हैं।",
    },
    { "en": "When in Rome, do as the Romans do.", "hi": "जैसा देश वैसा भेस।" },
    {
        "en": "You can lead a horse to water, but you can't make him drink.",
        "hi": "आप किसी को कुछ करने के लिए मजबूर नहीं कर सकते।",
    },
    {
        "en": "A watched pot never boils.",
        "hi": "किसी काम पर लगातार ध्यान देने से वह धीमा लगता है।",
    },
    { "en": "Better late than never.", "hi": "कभी न होने से देर भली।" },
    { "en": "Curiosity killed the cat.", "hi": "अधिक जिज्ञासा हानिकारक होती है।" },
    { "en": "He's such a drama queen.", "hi": "वह तो बस नौटंकी करता रहता है।" },
    { "en": "That's so fetch!", "hi": "यह तो बहुत अच्छा है!" },
    {
        "en": "I'm totally stoked about the concert.",
        "hi": "मैं कॉन्सर्ट के लिए बहुत उत्साहित हूँ।",
    },
    { "en": "She threw some shade at him.", "hi": "उसने उसकी बुराई की।" },
    {
        "en": "He's salty about not getting the promotion.",
        "hi": "वह प्रमोशन न मिलने से नाराज़ है।",
    },
    {
        "en": "Let's Netflix and chill.",
        "hi": "चलो साथ में फिल्म देखते हैं और आराम करते हैं।",
    },
    { "en": "That's my jam!", "hi": "यह मुझे बहुत पसंद है!" },
    { "en": "He's low-key a genius.", "hi": "वह अंदर ही अंदर एक जीनियस है।" },
    {
        "en": "I'm high-key annoyed right now.",
        "hi": "मैं इस समय बहुत ज़्यादा परेशान हूँ।",
    },
    { "en": "She's a total badass.", "hi": "वह बहुत दबंग है।" },
    { "en": "Don't be a basic.", "hi": "साधारण मत बनो।" },
    { "en": "He's ghosting me.", "hi": "वह मुझसे अचानक संपर्क तोड़ रहा है।" },
    { "en": "That's so cringe.", "hi": "यह बहुत शर्मनाक है।" },
    {
        "en": "I'm feeling boujee today.",
        "hi": "आज मैं थोड़ा ठाठ-बाट महसूस कर रहा हूँ।",
    },
    { "en": "He's a snack.", "hi": "वह बहुत आकर्षक है।" },
    { "en": "Let's get this bread.", "hi": "चलो पैसे कमाते हैं।" },
    { "en": "She's my ride or die.", "hi": "वह मेरी हर सुख-दुख की साथी है।" },
    { "en": "He's such a simp.", "hi": "वह किसी के लिए बहुत ज़्यादा उत्सुक है।" },
    { "en": "That's a mood.", "hi": "यह मेरी मनोदशा है।" },
    {
        "en": "I'm living my best life.",
        "hi": "मैं अपनी ज़िंदगी का सबसे अच्छा समय जी रहा हूँ।",
    },
    { "en": "He's really extra.", "hi": "वह बहुत दिखावटी है।" },
    { "en": "Spill the tea.", "hi": "गपशप बताओ।" },
    { "en": "No cap.", "hi": "सच कह रहा हूँ।" },
    { "en": "She's on fleek.", "hi": "वह बहुत अच्छी लग रही है।" },
    { "en": "He's salty.", "hi": "वह चिढ़ा हुआ है।" },
    { "en": "That's fire!", "hi": "यह बहुत शानदार है!" },
    { "en": "I'm so down for that.", "hi": "मैं इसके लिए पूरी तरह से तैयार हूँ।" },
    { "en": "He's a woke individual.", "hi": "वह एक जागरूक व्यक्ति है।" },
    { "en": "She's slaying!", "hi": "वह कमाल कर रही है!" },
    { "en": "That's sus.", "hi": "यह संदिग्ध है।" },
    { "en": "I'm yeeting out of here.", "hi": "मैं यहाँ से जा रहा हूँ।" },
    {
        "en": "He's a total himbo.",
        "hi": "वह दिखने में अच्छा लेकिन कम बुद्धिमान है।",
    },
    { "en": "She's serving looks.", "hi": "वह बहुत आकर्षक दिख रही है।" },
    { "en": "Don't @ me.", "hi": "मुझे टैग मत करो।" },
    { "en": "He's cancelled.", "hi": "उसका बहिष्कार किया जा रहा है।" },
    { "en": "That's a big yikes.", "hi": "यह बहुत बुरा है।" },
    {
        "en": "I'm feeling myself today.",
        "hi": "आज मैं खुद को बहुत अच्छा महसूस कर रहा हूँ।",
    },
    { "en": "He's a glow-up.", "hi": "उसमें बहुत सुधार आया है।" },
    {
        "en": "She's a Karen.",
        "hi": "वह एक ऐसी महिला है जो हर बात पर शिकायत करती है।",
    },
    { "en": "That's a vibe.", "hi": "यह एक अच्छा माहौल है।" },
    {
        "en": "I'm stanning this group.",
        "hi": "मैं इस समूह का बहुत बड़ा प्रशंसक हूँ।",
    },
    { "en": "He's a clout chaser.", "hi": "वह प्रसिद्धि पाने की कोशिश कर रहा है।" },
    { "en": "She's a soft girl.", "hi": "वह कोमल और संवेदनशील है।" },
    { "en": "That's a whole mood.", "hi": "यह पूरी तरह से मेरी मनोदशा है।" },
    { "en": "I'm simping hard.", "hi": "मैं किसी के लिए बहुत ज़्यादा उत्सुक हूँ।" },
    { "en": "He's a chad.", "hi": "वह आत्मविश्वास से भरा और आकर्षक पुरुष है।" },
    { "en": "She's a baddie.", "hi": "वह आत्मविश्वास से भरी और आकर्षक महिला है।" },
    { "en": "That's the tea.", "hi": "यह है असली बात।" },
    { "en": "I'm finna go.", "hi": "मैं जाने वाला हूँ।" },
    { "en": "He's boujee.", "hi": "वह दिखावटी है।" },
    { "en": "She's a savage.", "hi": "वह बहुत निर्दयी है।" },
    { "en": "That's a serve.", "hi": "यह बहुत प्रभावशाली है।" },
    { "en": "I'm feeling zesty.", "hi": "मैं उत्साहित महसूस कर रहा हूँ।" },
    { "en": "He's a stan.", "hi": "वह एक बहुत बड़ा प्रशंसक है।" },
    { "en": "She's a queen.", "hi": "वह बहुत शानदार है।" },
    { "en": "That's iconic.", "hi": "यह बहुत प्रसिद्ध है।" },
    { "en": "What's cooking?", "hi": "क्या बन रहा है?" },
    { "en": "Hang tight.", "hi": "थोड़ा रुको।" },
    { "en": "Bless your heart.", "hi": "भगवान भला करे।" },
    { "en": "That's sick!", "hi": "यह तो कमाल का है!" },
    { "en": "I'm beat.", "hi": "मैं थक गया हूँ।" },
    { "en": "Let's chill.", "hi": "चलो आराम करते हैं।" },
    { "en": "You bet!", "hi": "ज़रूर!" },
    { "en": "It cost an arm and a leg.", "hi": "यह बहुत महंगा पड़ा।" },
    { "en": "He's a night owl.", "hi": "वह देर रात तक जागने वाला है।" },
    { "en": "Cat got your tongue?", "hi": "क्या तुम्हारी ज़बान बंद हो गई है?" },
    { "en": "That's a load off my mind.", "hi": "यह सुनकर मुझे शांति मिली।" },
    { "en": "Butter someone up.", "hi": "किसी को मक्खन लगाना।" },
    { "en": "Call it quits.", "hi": "छोड़ देना।" },
    { "en": "Get off my back!", "hi": "मेरा पीछा छोड़ो!" },
    { "en": "Hold your horses.", "hi": "थोड़ा रुको।" },
    { "en": "Keep your fingers crossed.", "hi": "उम्मीद रखना।" },
    { "en": "Miss the boat.", "hi": "मौका चूक जाना।" },
    { "en": "On the ball.", "hi": "चुस्त और तैयार।" },
    { "en": "Pull yourself together.", "hi": "अपने आप को संभालो।" },
    { "en": "Speak of the devil!", "hi": "शैतान का नाम लिया और शैतान हाज़िर!" },
    {
        "en": "He's got his head in the clouds.",
        "hi": "वह ख्यालों की दुनिया में खोया रहता है।",
    },
    {
        "en": "Don't bite the hand that feeds you.",
        "hi": "जिस थाली में खाते हो, उसमें छेद मत करो।",
    },
    {
        "en": "I'm feeling a bit spaced out today.",
        "hi": "आज मेरा ध्यान थोड़ा भटक रहा है।",
    },
    { "en": "Let's take a rain check.", "hi": "चलो इसे फिर कभी करते हैं।" },
    {
        "en": "That's a moot point.",
        "hi": "यह एक विवादास्पद मुद्दा है जिस पर बहस करने का कोई फायदा नहीं।",
    },
    { "en": "It's like pulling teeth.", "hi": "यह बहुत मुश्किल काम है।" },
    {
        "en": "He's a diamond in the rough.",
        "hi": "उसमें प्रतिभा है, पर उसे तराशने की ज़रूरत है।",
    },
    {
        "en": "I'm feeling on edge.",
        "hi": "मैं थोड़ा घबराया हुआ महसूस कर रहा हूँ।",
    },
    {
        "en": "Let's agree to disagree.",
        "hi": "चलो इस बात पर सहमत होते हैं कि हम असहमत हैं।",
    },
    { "en": "Don't add fuel to the fire.", "hi": "आग में घी मत डालो।" },
    { "en": "He's a busy bee.", "hi": "वह बहुत व्यस्त रहता है।" },
    { "en": "Let's get down and dirty.", "hi": "चलो सीधे काम पर लग जाते हैं।" },
    { "en": "That's a long shot.", "hi": "उसकी संभावना बहुत कम है।" },
    { "en": "It's a slippery slope.", "hi": "यह एक खतरनाक राह है।" },
    { "en": "He's a lone wolf.", "hi": "वह अकेला रहना पसंद करता है।" },
    {
        "en": "I'm feeling a bit out of sorts.",
        "hi": "मैं थोड़ा असहज महसूस कर रहा हूँ।",
    },
    { "en": "Let's play it by ear.", "hi": "चलो देखते हैं आगे क्या होता है।" },
    {
        "en": "That's a tough pill to swallow.",
        "hi": "यह स्वीकार करना मुश्किल है।",
    },
    { "en": "Don't jump the gun.", "hi": "जल्दबाजी मत करो।" },
    { "en": "He's a walking encyclopedia.", "hi": "उसे बहुत ज्ञान है।" },
    {
        "en": "I'm feeling a bit off today.",
        "hi": "आज मेरी तबीयत थोड़ी ठीक नहीं है।",
    },
    { "en": "Let's touch base later.", "hi": "चलो बाद में बात करते हैं।" },
    { "en": "That's a wake-up call.", "hi": "यह एक चेतावनी है।" },
    {
        "en": "It's a win-win situation.",
        "hi": "यह दोनों पक्षों के लिए फायदेमंद है।",
    },
    { "en": "He's a chatterbox.", "hi": "वह बहुत बातूनी है।" },
    {
        "en": "I'm feeling a bit drained.",
        "hi": "मैं थोड़ा थका हुआ महसूस कर रहा हूँ।",
    },
    { "en": "Let's table this for now.", "hi": "चलो इसे अभी के लिए रोक देते हैं।" },
    { "en": "That's a can of worms.", "hi": "यह एक जटिल समस्या है।" },
    { "en": "Don't rock the boat.", "hi": "हालात मत बिगाड़ो।" },
    { "en": "He's a go-getter.", "hi": "वह मेहनती और महत्वाकांक्षी है।" },
    {
        "en": "I'm feeling a bit under pressure.",
        "hi": "मैं थोड़ा दबाव महसूस कर रहा हूँ।",
    },
    { "en": "Let's wrap things up.", "hi": "चलो सब कुछ समेटते हैं।" },
    { "en": "That's a ballpark figure.", "hi": "यह एक अनुमानित आंकड़ा है।" },
    { "en": "It's a learning curve.", "hi": "यह सीखने की प्रक्रिया है।" },
    { "en": "He's a mover and shaker.", "hi": "वह प्रभावशाली व्यक्ति है।" },
    {
        "en": "I'm feeling a bit overwhelmed.",
        "hi": "मैं थोड़ा अभिभूत महसूस कर रहा हूँ।",
    },
    {
        "en": "Let's circle back to this later.",
        "hi": "चलो बाद में इस पर फिर से बात करते हैं।",
    },
    { "en": "That's a game changer.", "hi": "यह सब कुछ बदल देगा।" },
    { "en": "He's an early bird.", "hi": "वह सुबह जल्दी उठने वाला है।" },
    {
        "en": "I'm feeling a bit restless.",
        "hi": "मैं थोड़ा बेचैन महसूस कर रहा हूँ।",
    },
    { "en": "Let's brainstorm some ideas.", "hi": "चलो कुछ विचार मंथन करते हैं।" },
    { "en": "That's a red flag.", "hi": "यह खतरे का संकेत है।" },
    { "en": "It's a tough crowd.", "hi": "यह मुश्किल लोगों का समूह है।" },
    { "en": "He's a smooth talker.", "hi": "वह मीठी-मीठी बातें करने वाला है।" },
    {
        "en": "I'm feeling a bit wired.",
        "hi": "मैं थोड़ा उत्तेजित महसूस कर रहा हूँ।",
    },
    { "en": "Let's get together sometime.", "hi": "चलो कभी मिलते हैं।" },
    { "en": "That's a golden opportunity.", "hi": "यह एक सुनहरा अवसर है।" },
    { "en": "Don't rain on my parade.", "hi": "मेरा मज़ा किरकिरा मत करो।" },
    { "en": "He's a workaholic.", "hi": "वह काम के प्रति बहुत समर्पित है।" },
    { "en": "I'm feeling a bit foggy.", "hi": "मेरा दिमाग थोड़ा धुंधला है।" },
    { "en": "Let's think outside the box.", "hi": "चलो कुछ अलग तरह से सोचते हैं।" },
    { "en": "That's a grey area.", "hi": "यह अस्पष्ट क्षेत्र है।" },
    { "en": "It's a long haul.", "hi": "यह एक लंबी प्रक्रिया है।" },
    { "en": "He's a team player.", "hi": "वह टीम में मिलकर काम करने वाला है।" },
    {
        "en": "I'm feeling a bit zonked.",
        "hi": "मैं बहुत थका हुआ महसूस कर रहा हूँ।",
    },
    {
        "en": "Let's play devil's advocate.",
        "hi": "चलो विपरीत दृष्टिकोण से बहस करते हैं।",
    },
    { "en": "That's a silver lining.", "hi": "यह मुश्किल में भी एक अच्छी बात है।" },
    {
        "en": "Don't bite off more than you can chew.",
        "hi": "अपनी क्षमता से ज़्यादा काम मत लो।",
    },
    { "en": "He's a breath of fresh air.", "hi": "वह ताजगी का एहसास कराता है।" },
    {
        "en": "I'm feeling a bit jittery.",
        "hi": "मैं थोड़ा घबराया हुआ महसूस कर रहा हूँ।",
    },
    {
        "en": "Let's get on the same page.",
        "hi": "चलो एक ही बात पर सहमत होते हैं।",
    },
    { "en": "That's a tough break.", "hi": "यह बहुत बुरा हुआ।" },
    { "en": "It's a vicious cycle.", "hi": "यह एक दुष्चक्र है।" },
    { "en": "He's a man of few words.", "hi": "वह कम बोलने वाला है।" },
    {
        "en": "I'm feeling a bit spaced out.",
        "hi": "आज मेरा ध्यान थोड़ा भटक रहा है।",
    },
    { "en": "Let's cut to the chase.", "hi": "सीधे मुद्दे पर आते हैं।" },
    { "en": "That's a recipe for disaster.", "hi": "यह बर्बादी का कारण बनेगा।" },
    {
        "en": "Don't beat yourself up about it.",
        "hi": "इसके बारे में खुद को दोष मत दो।",
    },
    { "en": "He's a people person.", "hi": "वह लोगों से आसानी से घुलमिल जाता है।" },
    {
        "en": "I'm feeling a bit drained.",
        "hi": "मैं थोड़ा थका हुआ महसूस कर रहा हूँ।",
    },
    { "en": "Let's think big.", "hi": "चलो बड़ा सोचते हैं।" },
    { "en": "That's a stepping stone.", "hi": "यह एक शुरुआती कदम है।" },
    { "en": "It's a whole new ballgame.", "hi": "यह बिल्कुल नई स्थिति है।" },
    { "en": "He's a trendsetter.", "hi": "वह नए चलन शुरू करने वाला है।" },
    {
        "en": "I'm feeling a bit out of my depth.",
        "hi": "मुझे यह थोड़ा मुश्किल लग रहा है।",
    },
    { "en": "Let's keep it simple.", "hi": "चलो इसे सरल रखते हैं।" },
    { "en": "Don't get your hopes up too high.", "hi": "ज़्यादा उम्मीदें मत रखो।" },
    { "en": "Let's go with the flow.", "hi": "चलो जैसे चल रहा है, चलने देते हैं।" },
    { "en": "That's a real bummer.", "hi": "यह सच में निराशाजनक है।" },
    { "en": "The whole nine yards.", "hi": "पूरी तरह से।" },
    {
        "en": "Bite off more than you can chew.",
        "hi": "अपनी क्षमता से ज़्यादा काम लेना।",
    },
    { "en": "Go the extra mile.", "hi": "अतिरिक्त प्रयास करना।" },
    { "en": "Hit the nail on the head.", "hi": "सही बात कहना।" },
    { "en": "Jump on the bandwagon.", "hi": "देखा-देखी करना।" },
    { "en": "Leave no stone unturned.", "hi": "हर संभव प्रयास करना।" },
    { "en": "Make a long story short.", "hi": "संक्षेप में कहना।" },
    { "en": "On the same page.", "hi": "एक ही राय होना।" },
    { "en": "Play it by ear.", "hi": "बिना तैयारी के काम करना।" },
    { "en": "Take a rain check.", "hi": "फिर कभी करना।" },
    {
        "en": "He's pulling my leg, isn't he?",
        "hi": "वह मेरी टांग खींच रहा है, है ना?",
    },
    {
        "en": "I'm feeling a bit out of sync with everyone today.",
        "hi": "आज मैं थोड़ा सबसे अलग महसूस कर रहा हूँ।",
    },
    {
        "en": "Let's not beat around the bush and get straight to the point.",
        "hi": "चलो गोलमोल बातें नहीं करते और सीधे मुद्दे पर आते हैं।",
    },
    {
        "en": "That situation was a real Catch-22; whatever we did, we'd be in trouble.",
        "hi": "वह स्थिति सच में एक कैच-22 थी; हम कुछ भी करते, मुसीबत में पड़ते।",
    },
    {
        "en": "She's got a bee in her bonnet about the new regulations.",
        "hi": "नई नियमों को लेकर उसके दिमाग में कुछ परेशानी चल रही है।",
    },
    {
        "en": "He's always trying to cut corners, which often leads to more problems down the line.",
        "hi": "वह हमेशा शॉर्टकट मारने की कोशिश करता है, जिससे अक्सर आगे चलकर और ज़्यादा समस्याएँ आती हैं।",
    },
    {
        "en": "If you don't get your ducks in a row, this project is going to fall apart.",
        "hi": "अगर तुम सब कुछ ठीक से व्यवस्थित नहीं करोगे, तो यह प्रोजेक्ट बिगड़ जाएगा।",
    },
    {
        "en": "Despite the initial setbacks, they managed to turn the tables and come out on top.",
        "hi": "शुरुआती झटकों के बावजूद, वे स्थिति पलटने और शीर्ष पर आने में कामयाब रहे।",
    },
    {
        "en": "I'm walking on eggshells around him because he's been so touchy lately.",
        "hi": "मैं उसके आसपास बहुत सावधानी से पेश आ रहा हूँ क्योंकि वह आजकल बहुत चिड़चिड़ा है।",
    },
    {
        "en": "Let's just agree to disagree and move on to a different topic.",
        "hi": "चलो बस इस बात पर असहमत होते हैं और किसी और विषय पर चलते हैं।",
    },
    {
        "en": "He spilled the beans about the surprise party, so now it's not a surprise anymore.",
        "hi": "उसने सरप्राइज पार्टी के बारे में सब बता दिया, इसलिए अब यह सरप्राइज नहीं रही।",
    },
    {
        "en": "She's burning the midnight oil to finish the report before the deadline.",
        "hi": "वह डेडलाइन से पहले रिपोर्ट खत्म करने के लिए देर रात तक काम कर रही है।",
    },
    {
        "en": "Don't put all your eggs in one basket; it's always good to have a backup plan.",
        "hi": "अपना सब कुछ एक ही जगह मत लगाओ; हमेशा एक बैकअप योजना रखना अच्छा होता है।",
    },
    {
        "en": "He's been feeling under the weather for a few days, so he decided to stay home.",
        "hi": "वह कुछ दिनों से ठीक महसूस नहीं कर रहा था, इसलिए उसने घर पर रहने का फैसला किया।",
    },
    {
        "en": "They decided to bury the hatchet after years of feuding.",
        "hi": "उन्होंने सालों की दुश्मनी के बाद सुलह करने का फैसला किया।",
    },
    {
        "en": "It's a dog-eat-dog world out there; you have to be tough to survive.",
        "hi": "बाहर बहुत प्रतिस्पर्धा है; जीवित रहने के लिए आपको मजबूत होना होगा।",
    },
    {
        "en": "He's a jack of all trades but master of none.",
        "hi": "वह सब काम कर लेता है लेकिन किसी में माहिर नहीं है।",
    },
    {
        "en": "I'm at my wit's end trying to solve this problem.",
        "hi": "मैं इस समस्या को हल करने की कोशिश करते-करते परेशान हो गया हूँ।",
    },
    {
        "en": "Let's get our ducks in a row before we launch the new product.",
        "hi": "नया उत्पाद लॉन्च करने से पहले चलो सब कुछ व्यवस्थित कर लेते हैं।",
    },
    {
        "en": "That's a tough act to follow after such an amazing performance.",
        "hi": "इतने शानदार प्रदर्शन के बाद यह एक मुश्किल काम है।",
    },
    {
        "en": "He's always trying to have his cake and eat it too.",
        "hi": "वह हमेशा दोनों तरफ से फायदा उठाना चाहता है।",
    },
    {
        "en": "I'm feeling a bit out of my element in this new role.",
        "hi": "मैं इस नई भूमिका में थोड़ा असहज महसूस कर रहा हूँ।",
    },
    {
        "en": "Let's not jump the gun and wait for all the information before making a decision.",
        "hi": "चलो जल्दबाजी नहीं करते और फैसला करने से पहले सारी जानकारी का इंतजार करते हैं।",
    },
    {
        "en": "That project turned out to be a real white elephant; it cost a lot but had no real value.",
        "hi": "वह प्रोजेक्ट सच में एक बेकार की चीज साबित हुआ; उसमें बहुत खर्च आया लेकिन उसका कोई वास्तविक मूल्य नहीं था।",
    },
    {
        "en": "She's been bending over backwards to help everyone.",
        "hi": "वह सबकी मदद करने के लिए बहुत प्रयास कर रही है।",
    },
    {
        "en": "He's a dark horse in this competition; no one expects him to win.",
        "hi": "वह इस प्रतियोगिता में छुपा रुस्तम है; किसी को उससे जीतने की उम्मीद नहीं है।",
    },
    {
        "en": "I'm feeling a bit under the weather, so I might take it easy today.",
        "hi": "मेरी तबीयत थोड़ी नासाज है, इसलिए शायद मैं आज आराम करूँ।",
    },
    {
        "en": "Let's call a spade a spade and be honest about the situation.",
        "hi": "चलो सीधी बात करते हैं और स्थिति के बारे में ईमानदार रहते हैं।",
    },
    {
        "en": "That was a wild goose chase; we spent hours looking for something that wasn't there.",
        "hi": "वह एक बेकार की खोज थी; हमने घंटों तक ऐसी चीज की तलाश में बिताए जो वहाँ थी ही नहीं।",
    },
    {
        "en": "He's a man of his word; if he says he'll do something, he'll do it.",
        "hi": "वह अपनी बात का पक्का है; अगर वह कहता है कि वह कुछ करेगा, तो वह करेगा।",
    },
    {
        "en": "I'm feeling a bit peckish; maybe we should grab a snack.",
        "hi": "मुझे थोड़ी भूख लगी है; शायद हमें कुछ नाश्ता कर लेना चाहिए।",
    },
    {
        "en": "Let's hit the hay; it's getting late.",
        "hi": "चलो सोने चलते हैं; बहुत देर हो रही है।",
    },
    {
        "en": "That car was a real steal; he got it for a fraction of its actual price.",
        "hi": "वह कार सच में बहुत सस्ती थी; उसने उसे उसकी असली कीमत के एक छोटे से हिस्से में खरीदा।",
    },
    {
        "en": "Don't bite off more than you can chew, especially with such a complex task.",
        "hi": "अपनी क्षमता से ज़्यादा काम मत लो, खासकर इतने जटिल काम के साथ।",
    },
    {
        "en": "He's a big fish in a small pond here, but in the larger industry, he's not as significant.",
        "hi": "यहाँ वह छोटे लोगों में बड़ा आदमी है, लेकिन बड़े उद्योग में वह उतना महत्वपूर्ण नहीं है।",
    },
    {
        "en": "I'm caught between a rock and a hard place; I don't know which option to choose.",
        "hi": "मैं मुश्किल में हूँ; मुझे नहीं पता कि कौन सा विकल्प चुनना है।",
    },
    {
        "en": "Let's cut to the chase; what's the bottom line?",
        "hi": "सीधे मुद्दे पर आते हैं; आखिर बात क्या है?",
    },
    {
        "en": "That's a long shot, but it's worth trying.",
        "hi": "उसकी संभावना बहुत कम है, लेकिन कोशिश करने लायक है।",
    },
    {
        "en": "It's all Greek to me; I have no idea what you're talking about.",
        "hi": "यह मेरी समझ से परे है; मुझे कोई अंदाज़ा नहीं है कि तुम क्या कह रहे हो।",
    },
    {
        "en": "He's a dark horse in the race for the promotion.",
        "hi": "वह प्रमोशन की दौड़ में छुपा रुस्तम है।",
    },
    {
        "en": "I'm feeling on top of the world after hearing the good news.",
        "hi": "अच्छी खबर सुनने के बाद मैं बहुत खुश महसूस कर रहा हूँ।",
    },
    {
        "en": "Let's play it by ear and see how things develop.",
        "hi": "चलो देखते हैं आगे क्या होता है और उसके अनुसार काम करते हैं।",
    },
    {
        "en": "That's a tough pill to swallow, but we have to accept it.",
        "hi": "यह मानना मुश्किल है, लेकिन हमें इसे स्वीकार करना होगा।",
    },
    {
        "en": "Don't put all your eggs in one basket when it comes to investments.",
        "hi": "जब निवेश की बात आती है तो अपना सब कुछ एक ही जगह मत लगाओ।",
    },
    {
        "en": "He's a wolf in sheep's clothing; don't be fooled by his innocent appearance.",
        "hi": "वह धोखेबाज है; उसकी मासूमियत से धोखा मत खाना।",
    },
    {
        "en": "I'm feeling a bit under the weather; I think I might have caught a cold.",
        "hi": "मेरी तबीयत थोड़ी खराब लग रही है; मुझे लगता है कि मुझे सर्दी लग गई है।",
    },
    {
        "en": "Let's get this show on the road; we're already running late.",
        "hi": "चलो शुरू करते हैं; हमें पहले ही देर हो रही है।",
    },
    {
        "en": "That's a red herring; it's meant to distract us from the real issue.",
        "hi": "यह भटकाने वाली बात है; इसका मकसद हमें असली मुद्दे से ध्यान हटाना है।",
    },
    {
        "en": "It's a vicious cycle; one problem leads to another.",
        "hi": "यह एक दुष्चक्र है; एक समस्या दूसरी समस्या को जन्म देती है।",
    },
    {
        "en": "He's a yes-man; he agrees with everything the boss says.",
        "hi": "वह हर बात में हाँ में हाँ मिलाता है; वह बॉस की हर बात से सहमत होता है।",
    },
    {
        "en": "I'm feeling a bit out of sorts; I can't quite put my finger on it.",
        "hi": "मैं थोड़ा परेशान महसूस कर रहा हूँ; मैं ठीक से बता नहीं सकता क्या है।",
    },
    {
        "en": "Let's bury the hatchet and move forward peacefully.",
        "hi": "चलो दुश्मनी भुला देते हैं और शांति से आगे बढ़ते हैं।",
    },
    {
        "en": "That old car is a real white elephant; it's expensive to maintain and doesn't get much use.",
        "hi": "वह पुरानी कार सच में एक बेकार की चीज है; उसका रखरखाव महंगा है और वह ज़्यादा इस्तेमाल नहीं होती।",
    },
    {
        "en": "Don't count your chickens before they hatch; anything can happen.",
        "hi": "सपने सच होने से पहले उन पर भरोसा मत करो; कुछ भी हो सकता है।",
    },
    {
        "en": "He's a wet blanket; he always ruins the fun.",
        "hi": "वह मजा किरकिरा करने वाला है; वह हमेशा मज़ा खराब कर देता है।",
    },
    {
        "en": "I'm feeling a bit off-color today; I think I need some rest.",
        "hi": "आज मेरी तबीयत थोड़ी ठीक नहीं है; मुझे लगता है मुझे आराम की ज़रूरत है।",
    },
    {
        "en": "Let's call a spade a spade and address the real issues.",
        "hi": "चलो सीधी बात करते हैं और असली मुद्दों का समाधान करते हैं।",
    },
    {
        "en": "That was a wild goose chase; we ended up nowhere.",
        "hi": "वह एक बेकार की खोज थी; हम कहीं नहीं पहुँचे।",
    },
    {
        "en": "It's a dog-eat-dog world, especially in the competitive job market.",
        "hi": "यह बहुत प्रतिस्पर्धी दुनिया है, खासकर प्रतिस्पर्धी नौकरी बाजार में।",
    },
    {
        "en": "He's a jack-of-all-trades, so he can probably fix it.",
        "hi": "वह सब काम कर लेता है, इसलिए शायद वह इसे ठीक कर देगा।",
    },
    {
        "en": "I'm feeling a bit down in the dumps today for no particular reason.",
        "hi": "आज मैं बिना किसी खास कारण के थोड़ा उदास महसूस कर रहा हूँ।",
    },
    {
        "en": "Let's clear the air and talk about what's really bothering us.",
        "hi": "चलो गलतफहमी दूर करते हैं और बात करते हैं कि हमें वास्तव में क्या परेशान कर रहा है।",
    },
    {
        "en": "That's a tall order; I'm not sure if we can achieve that.",
        "hi": "यह मुश्किल काम है; मुझे नहीं पता कि क्या हम उसे हासिल कर सकते हैं।",
    },
    {
        "en": "Don't put off until tomorrow what you can do today; procrastination is the thief of time.",
        "hi": "जो काम आज कर सकते हो, उसे कल पर मत टालो; टालमटोल समय का चोर है।",
    },
    {
        "en": "He's a rolling stone; he never stays in one place for too long.",
        "hi": "वह एक जगह नहीं टिकता; वह कभी भी एक जगह पर ज़्यादा देर तक नहीं रहता।",
    },
    {
        "en": "I'm feeling a bit under the weather; I hope it's nothing serious.",
        "hi": "मेरी तबीयत थोड़ी खराब है; मुझे उम्मीद है कि यह कुछ गंभीर नहीं है।",
    },
    {
        "en": "Let's go Dutch; everyone pays for their own meal.",
        "hi": "चलो अपना-अपना खर्चा बाँट लेते हैं; हर कोई अपने खाने का भुगतान करेगा।",
    },
    {
        "en": "That's a shot in the dark, but it's worth a try.",
        "hi": "यह बिना सोचे-समझे किया गया काम है, लेकिन कोशिश करने लायक है।",
    },
    {
        "en": "It's a double-edged sword; it has both advantages and disadvantages.",
        "hi": "यह दोधारी तलवार है; इसके फायदे और नुकसान दोनों हैं।",
    },
    {
        "en": "He's a loose cannon; you never know what he's going to do next.",
        "hi": "वह अनियंत्रित है; आपको कभी नहीं पता चलेगा कि वह आगे क्या करने वाला है।",
    },
    {
        "en": "I'm feeling a bit out of it today; I didn't sleep well last night.",
        "hi": "आज मैं थोड़ा सुस्त महसूस कर रहा हूँ; मैं कल रात अच्छी तरह से नहीं सोया।",
    },
    {
        "en": "Let's get our ducks in a row before the meeting starts.",
        "hi": "बैठक शुरू होने से पहले चलो सब कुछ व्यवस्थित कर लेते हैं।",
    },
    {
        "en": "That was a slap in the face; I didn't expect such harsh criticism.",
        "hi": "यह अपमानजनक था; मुझे इतनी कड़ी आलोचना की उम्मीद नहीं थी।",
    },
    {
        "en": "Don't make a mountain out of a molehill; it's not that big of a deal.",
        "hi": "बात का बतंगड़ मत बनाओ; यह इतनी बड़ी बात नहीं है।",
    },
    {
        "en": "He's a pain in the neck; he's always causing trouble.",
        "hi": "वह सिरदर्द है; वह हमेशा परेशानी पैदा करता रहता है।",
    },
    {
        "en": "I'm feeling a bit under the gun with all these deadlines.",
        "hi": "मैं इन सभी डेडलाइन के साथ थोड़ा दबाव में महसूस कर रहा हूँ।",
    },
    {
        "en": "Let's go the extra mile to ensure customer satisfaction.",
        "hi": "चलो ग्राहक संतुष्टि सुनिश्चित करने के लिए अतिरिक्त प्रयास करते हैं।",
    },
    {
        "en": "That's a steal of a deal; you should definitely go for it.",
        "hi": "यह बहुत सस्ता सौदा है; तुम्हें निश्चित रूप से इसके लिए जाना चाहिए।",
    },
    {
        "en": "It turned out to be a blessing in disguise; the initial problem led to a better outcome.",
        "hi": "यह अप्रत्याशित रूप से अच्छा साबित हुआ; शुरुआती समस्या एक बेहतर परिणाम की ओर ले गई।",
    },
    {
        "en": "He's such a couch potato; he spends all his time watching TV.",
        "hi": "वह तो बस घर में पड़ा रहता है; वह अपना सारा समय टीवी देखने में बिताता है।",
    },
    {
        "en": "I'm feeling a bit off-color; I think I need to rest and drink some fluids.",
        "hi": "आज मेरी तबीयत थोड़ी ठीक नहीं है; मुझे लगता है मुझे आराम करने और कुछ तरल पदार्थ पीने की ज़रूरत है।",
    },
    {
        "en": "Let's get our hands dirty and start working on this project.",
        "hi": "चलो काम में लग जाते हैं और इस प्रोजेक्ट पर काम करना शुरू करते हैं।",
    },
    {
        "en": "That was a bitter pill to swallow, but we had no other choice.",
        "hi": "यह स्वीकार करना मुश्किल था, लेकिन हमारे पास कोई और विकल्प नहीं था।",
    },
    {
        "en": "Don't burn your bridges; you might need those connections in the future.",
        "hi": "अपने संबंध मत बिगाड़ो; तुम्हें भविष्य में उन कनेक्शनों की आवश्यकता हो सकती है।",
    },
    {
        "en": "He's a chip off the old block; he's just like his father.",
        "hi": "वह अपने माता-पिता जैसा है; वह बिल्कुल अपने पिता जैसा है।",
    },
    {
        "en": "I'm feeling a bit blue today; I miss my family.",
        "hi": "आज मैं थोड़ा उदास महसूस कर रहा हूँ; मुझे अपने परिवार की याद आ रही है।",
    },
    { "en": "Get real.", "hi": "असली बात करो।" },
    { "en": "No biggie.", "hi": "कोई बड़ी बात नहीं।" },
    { "en": "Suit yourself.", "hi": "जैसी तुम्हारी मर्जी।" },
    { "en": "That's a wrap.", "hi": "बस हो गया।" },
    { "en": "What a drag!", "hi": "कितनी बोरिंग बात है!" },
    { "en": "You nailed it!", "hi": "तुमने कमाल कर दिया!" },
    { "en": "Better safe than sorry.", "hi": "चूक से बचाव बेहतर।" },
    { "en": "Give me a break!", "hi": "मुझे बख्शो!" },
    { "en": "It's not my forte.", "hi": "यह मेरा काम नहीं है।" },
    { "en": "Let's face it.", "hi": "चलो सच का सामना करते हैं।" },
    { "en": "That's a tough one.", "hi": "यह मुश्किल है।" },
    { "en": "What's the catch?", "hi": "इसमें क्या चाल है?" },
    { "en": "You're telling me!", "hi": "तुम सही कह रहे हो!" },
    { "en": "Couldn't be better.", "hi": "इससे बेहतर नहीं हो सकता।" },
    { "en": "Easy does it.", "hi": "धीरे-धीरे करो।" },
    { "en": "Get a grip.", "hi": "खुद को संभालो।" },
    { "en": "I'm all ears.", "hi": "मैं ध्यान से सुन रहा हूँ।" },
    { "en": "Long story short.", "hi": "संक्षेप में।" },
    { "en": "No sweat.", "hi": "कोई परेशानी नहीं।" },
    { "en": "That's the spirit!", "hi": "यह हुई ना बात!" },
    { "en": "He's got a point.", "hi": "उसकी बात में दम है।" },
    { "en": "I'm drawing a blank.", "hi": "मुझे कुछ याद नहीं आ रहा।" },
    { "en": "Let's play it safe.", "hi": "चलो सुरक्षित खेलते हैं।" },
    { "en": "That's beside the point.", "hi": "यह मुद्दे से अलग है।" },
    { "en": "What's the deal with that?", "hi": "उसका क्या चक्कर है?" },
    {
        "en": "You're barking up the wrong tree.",
        "hi": "तुम गलत जगह कोशिश कर रहे हो।",
    },
    {
        "en": "A bird in the hand is worth two in the bush.",
        "hi": "हाथ का कंगन क्या देखना।",
    },
    { "en": "Don't put the cart before the horse.", "hi": "उल्टा काम मत करो।" },
    { "en": "Every dog has its day.", "hi": "सबका समय आता है।" },
    { "en": "Look what the cat dragged in!", "hi": "यह कौन आ गया!" },
    { "en": "That's a fishy story.", "hi": "यह कहानी संदिग्ध है।" },
    { "en": "Water under the bridge.", "hi": "जो हो गया सो हो गया।" },
    {
        "en": "You can't have your cake and eat it too.",
        "hi": "तुम दोनों नहीं पा सकते।",
    },
    { "en": "A leopard can't change its spots.", "hi": "आदत कभी नहीं बदलती।" },
    {
        "en": "Don't cry over spilt milk.",
        "hi": "अब पछताए होत क्या जब चिड़िया चुग गई खेत।",
    },
    { "en": "Great minds think alike.", "hi": "समझदार लोग एक जैसा सोचते हैं।" },
    {
        "en": "The early bird gets the worm.",
        "hi": "जल्दी करने वाले को फायदा होता है।",
    },
    {
        "en": "When it rains, it pours.",
        "hi": "जब मुसीबत आती है, तो सब तरफ से आती है।",
    },
    { "en": "You live and learn.", "hi": "ज़िंदगी सीख देती है।" },
    { "en": "He's got sticky fingers.", "hi": "उसकी चोरी करने की आदत है।" },
    { "en": "I'm in a tight spot.", "hi": "मैं मुश्किल में हूँ।" },
    { "en": "Let's get the show on the road.", "hi": "चलो शुरू करते हैं।" },
    { "en": "That's a hard pill to swallow.", "hi": "यह मानना मुश्किल है।" },
    { "en": "What's eating you?", "hi": "तुम्हें क्या परेशान कर रहा है?" },
    { "en": "You're pulling my chain.", "hi": "तुम मुझसे मजाक कर रहे हो।" },
    { "en": "Beggars can't be choosers.", "hi": "मांगने वालों की पसंद नहीं होती।" },
    { "en": "Give it your best shot.", "hi": "पूरी कोशिश करो।" },
    { "en": "It takes two to tango.", "hi": "ताली दोनों हाथों से बजती है।" },
    { "en": "Look before you leap.", "hi": "सोच समझकर कदम उठाओ।" },
    { "en": "That's a moot point now.", "hi": "अब उस बात का कोई मतलब नहीं।" },
    { "en": "What's the bottom line?", "hi": "आखिर बात क्या है?" },
    { "en": "You've hit the jackpot!", "hi": "तुम्हारी तो लॉटरी लग गई!" },
    {
        "en": "Don't judge a book by its cover.",
        "hi": "किसी को बाहरी रूप से मत आंकना।",
    },
    { "en": "Old habits die hard.", "hi": "पुरानी आदतें मुश्किल से छूटती हैं।" },
    {
        "en": "I'm feeling a bit under the weather today.",
        "hi": "आज मेरी तबीयत थोड़ी नासाज है।",
    },
    {
        "en": "It's always darkest before the dawn.",
        "hi": "बुरा समय हमेशा अच्छे समय से पहले आता है।",
    },
    { "en": "Out of sight, out of mind.", "hi": "आँख ओझल, मन ओझल।" },
    { "en": "The best of both worlds.", "hi": "दोनों तरफ से फायदा।" },
    { "en": "I'm walking on air.", "hi": "मैं बहुत खुश हूँ।" },
    { "en": "You've got another think coming.", "hi": "तुम गलत सोच रहे हो।" },
    {
        "en": "You can't make an omelet without breaking eggs.",
        "hi": "कुछ पाने के लिए कुछ खोना पड़ता है।",
    },
    { "en": "That's a storm in a teacup.", "hi": "यह छोटी बात का बतंगड़ है।" },

    { "en": "Honesty is the best policy.", "hi": "ईमानदारी सबसे अच्छी नीति है।" },
    { "en": "It's the best of both worlds.", "hi": "दोनों तरफ से फायदा।" },
    {
        "en": "I'm just chilling at home, watching Netflix and doing nothing.",
        "hi": "मैं बस घर पर चिल कर रहा हूँ, नेटफ्लिक्स देख रहा हूँ और कुछ नहीं कर रहा।",
    },
    {
        "en": "She's such a drama queen; everything is a huge deal with her.",
        "hi": "वह तो एकदम नौटंकी है; उसके लिए हर बात बहुत बड़ी होती है।",
    },
    {
        "en": "He's really down to earth, despite being so famous.",
        "hi": "इतना मशहूर होने के बावजूद, वह सच में बहुत ज़मीनी इंसान है।",
    },
    {
        "en": "I can't even; this is just too much to handle.",
        "hi": "मुझसे तो अब नहीं होगा; यह सब झेलना बहुत मुश्किल है।",
    },
    {
        "en": "She's always throwing shade at everyone; so negative.",
        "hi": "वह हमेशा सब पर कटाक्ष करती रहती है; बहुत नकारात्मक है।",
    },
    {
        "en": "I'm feeling kind of blah today; no energy.",
        "hi": "आज मैं थोड़ा उदास महसूस कर रहा हूँ; कोई ऊर्जा नहीं है।",
    },
    {
        "en": "Let's bounce; this place is boring.",
        "hi": "चलो चलते हैं; यह जगह उबाऊ है।",
    },
    {
        "en": "He's living rent-free in my head; I can't stop thinking about it.",
        "hi": "वह मेरे दिमाग में मुफ्त में रह रहा है; मैं इसके बारे में सोचना बंद नहीं कर सकता।",
    },
    {
        "en": "That's cap; he's totally lying.",
        "hi": "वह झूठ है; वह पूरी तरह से झूठ बोल रहा है।",
    },
    {
        "en": "She's a real one; always there for her friends.",
        "hi": "वह सच्ची दोस्त है; हमेशा अपने दोस्तों के लिए मौजूद रहती है।",
    },
    {
        "en": "He's so extra; always doing the most unnecessary things.",
        "hi": "वह बहुत ज़्यादा है; हमेशा सबसे अनावश्यक चीजें करता है।",
    },
    {
        "en": "I'm just trying to stay in my lane and not get involved.",
        "hi": "मैं बस अपनी हद में रहने और शामिल न होने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "That's the tea; now you know the gossip.",
        "hi": "यह है असली बात; अब तुम्हें सारी गपशप पता है।",
    },
    {
        "en": "He's such a mood; sometimes happy, sometimes sad.",
        "hi": "उसका तो अलग ही मूड है; कभी खुश, कभी उदास।",
    },
    {
        "en": "I'm trying to manifest good things into my life.",
        "hi": "मैं अपने जीवन में अच्छी चीजों को आकर्षित करने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "That's a major glow-up; she looks amazing.",
        "hi": "यह तो ज़बरदस्त बदलाव है; वह बहुत खूबसूरत लग रही है।",
    },
    {
        "en": "He's always chasing clout; so desperate for attention.",
        "hi": "वह हमेशा प्रसिद्धि के पीछे भागता रहता है; ध्यान पाने के लिए बहुत बेताब है।",
    },
    {
        "en": "I'm just trying to keep it 100; being honest.",
        "hi": "मैं बस सच बोल रहा हूँ; ईमानदार रह रहा हूँ।",
    },
    {
        "en": "That's a vibe; I like the atmosphere here.",
        "hi": "यह एक अच्छा माहौल है; मुझे यहाँ का वातावरण पसंद है।",
    },
    {
        "en": "He's a snake; can't trust him at all.",
        "hi": "वह एक साँप है; उस पर बिल्कुल भरोसा नहीं किया जा सकता।",
    },
    {
        "en": "Let's link up later; we need to catch up.",
        "hi": "चलो बाद में मिलते हैं; हमें बातचीत करनी है।",
    },
    {
        "en": "That's a whole other level; way beyond what I expected.",
        "hi": "यह तो बिल्कुल अलग स्तर का है; मेरी उम्मीद से कहीं ज़्यादा।",
    },
    {
        "en": "He's such a soft boi; so gentle and sensitive.",
        "hi": "वह तो एकदम नरम दिल का लड़का है; बहुत कोमल और संवेदनशील।",
    },
    {
        "en": "I'm just trying to stay positive and keep my chin up.",
        "hi": "मैं बस सकारात्मक रहने और हिम्मत बनाए रखने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "That's a big yikes; that's really embarrassing.",
        "hi": "यह तो बहुत शर्मनाक है।",
    },
    {
        "en": "I'm just trying to live my best life.",
        "hi": "मैं बस अपनी ज़िंदगी का बेहतरीन तरीके से जीने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "That's a total mood killer; ruined the fun.",
        "hi": "यह तो पूरी तरह से मूड खराब करने वाला था; मज़ा किरकिरा कर दिया।",
    },
    {
        "en": "He's such a hypebeast; obsessed with trendy clothes.",
        "hi": "वह तो एकदम फैशन का कीड़ा है; ट्रेंडी कपड़ों का दीवाना है।",
    },
    {
        "en": "I'm just trying to keep it real; being authentic.",
        "hi": "मैं बस असली रहने की कोशिश कर रहा हूँ; वास्तविक बन रहा हूँ।",
    },
    {
        "en": "That's a major flex; showing off a lot.",
        "hi": "यह तो बहुत बड़ा दिखावा है; बहुत ज़्यादा शेखी बघार रहा है।",
    },
    {
        "en": "He's such a himbo; attractive but not very bright.",
        "hi": "वह तो एकदम सीधा-सादा दिखने वाला है; आकर्षक लेकिन ज़्यादा बुद्धिमान नहीं।",
    },
    {
        "en": "I'm just trying to go with the flow and see what happens.",
        "hi": "मैं बस जैसा चल रहा है, चलने देने और देखने की कोशिश कर रहा हूँ कि क्या होता है।",
    },
    {
        "en": "That's a whole vibe; I'm really feeling it.",
        "hi": "यह तो कमाल का माहौल है; मैं इसे सच में महसूस कर रहा हूँ।",
    },
    {
        "en": "He's such a boomer; stuck in old ways of thinking.",
        "hi": "वह तो एकदम पुराने ख्यालों का आदमी है; सोचने के पुराने तरीकों में फंसा हुआ है।",
    },
    {
        "en": "I'm just trying to get my life together.",
        "hi": "मैं बस अपनी ज़िंदगी को व्यवस्थित करने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "He's such a stan; a really enthusiastic fan.",
        "hi": "वह तो एकदम कट्टर प्रशंसक है; बहुत उत्साही फैन।",
    },
    {
        "en": "That's a big no-no; definitely not allowed.",
        "hi": "यह तो बिल्कुल मना है; बिल्कुल भी अनुमति नहीं है।",
    },
    {
        "en": "He's such a woke person; very aware of social issues.",
        "hi": "वह तो बहुत जागरूक व्यक्ति है; सामाजिक मुद्दों के बारे में बहुत सचेत है।",
    },
    {
        "en": "I'm just trying to stay hydrated and drink more water.",
        "hi": "मैं बस खुद को हाइड्रेटेड रखने और ज़्यादा पानी पीने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "That's a total facepalm moment; so embarrassing.",
        "hi": "यह तो एकदम सिर पकड़ लेने वाला पल है; बहुत शर्मनाक।",
    },
    {
        "en": "He's such a foodie; loves trying new restaurants.",
        "hi": "वह तो एकदम खाने का शौकीन है; नए रेस्तरां आज़माना पसंद करता है।",
    },
    {
        "en": "I'm just trying to get through the day.",
        "hi": "मैं बस दिन गुजारने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "That's a silver bullet; a perfect solution.",
        "hi": "यह तो अचूक उपाय है; एक एकदम सही समाधान।",
    },
    {
        "en": "I'm just trying to focus on my goals.",
        "hi": "मैं बस अपने लक्ष्यों पर ध्यान केंद्रित करने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "That's a mic drop moment; a perfect ending.",
        "hi": "यह तो शानदार अंत है; एक एकदम सही समाप्ति।",
    },
    {
        "en": "I'm just trying to be more mindful.",
        "hi": "मैं बस और ज़्यादा जागरूक रहने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "That's a major key; an important piece of advice.",
        "hi": "यह तो बहुत ज़रूरी बात है; सलाह का एक महत्वपूर्ण हिस्सा।",
    },
    {
        "en": "He's such a gym rat; always working out.",
        "hi": "वह तो एकदम जिम का कीड़ा है; हमेशा कसरत करता रहता है।",
    },
    {
        "en": "I'm just trying to find my peace.",
        "hi": "मैं बस अपनी शांति खोजने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "That's a game changer; it will change everything.",
        "hi": "यह तो सब कुछ बदल देगा।",
    },
    {
        "en": "He's such a bookworm; always reading.",
        "hi": "वह तो एकदम किताबी कीड़ा है; हमेशा पढ़ता रहता है।",
    },
    {
        "en": "That's a total vibe check; how are you feeling?",
        "hi": "यह तो माहौल का जायज़ा है; तुम कैसा महसूस कर रहे हो?.",
    },
    {
        "en": "He's such a night owl; always up late.",
        "hi": "वह तो रात का उल्लू है; हमेशा देर रात तक जागता रहता है।",
    },
    {
        "en": "I'm just trying to learn and grow.",
        "hi": "मैं बस सीखने और आगे बढ़ने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "That's a whole mood; I totally relate.",
        "hi": "यह तो बिल्कुल मेरा हाल है; मैं पूरी तरह से समझ सकता हूँ।",
    },
    {
        "en": "I'm just trying to be myself.",
        "hi": "मैं बस खुद बनने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "He's such a drama llama; always causing trouble.",
        "hi": "वह तो एकदम झगड़ालू है; हमेशा परेशानी खड़ी करता है।",
    },
    {
        "en": "I'm just trying to take things one day at a time.",
        "hi": "मैं बस एक-एक दिन करके चीज़ों को लेने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "That's a whole different ballgame; things are very different now.",
        "hi": "यह तो बिल्कुल अलग मामला है; अब चीजें बहुत अलग हैं।",
    },
    {
        "en": "I'm just trying to enjoy the moment.",
        "hi": "मैं बस इस पल का आनंद लेने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "That's a major throwback; brings back old memories.",
        "hi": "यह तो पुरानी यादें ताज़ा कर देने वाला है।",
    },
    {
        "en": "He's such a chatterbox; talks incessantly.",
        "hi": "वह तो बहुत बातूनी है; लगातार बोलता रहता है।",
    },
    {
        "en": "I'm just trying to get some sleep.",
        "hi": "मैं बस थोड़ी नींद लेने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "That's a tough crowd; difficult to please.",
        "hi": "यह मुश्किल लोगों का समूह है; खुश करना मुश्किल है।",
    },
    {
        "en": "He's such a people person; good at socializing.",
        "hi": "वह लोगों से आसानी से घुलमिल जाता है।",
    },
    {
        "en": "That's a golden opportunity; don't miss it.",
        "hi": "यह एक सुनहरा अवसर है; इसे मत गंवाना।",
    },
    {
        "en": "He's such a workaholic; always working.",
        "hi": "वह काम के प्रति बहुत समर्पित है; हमेशा काम करता रहता है।",
    },
    {
        "en": "I'm just trying to relax.",
        "hi": "मैं बस आराम करने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "I'm just trying to be happy.",
        "hi": "मैं बस खुश रहने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "I'm just winging it; I have no idea what I'm doing.",
        "hi": "मैं बस अंदाज़े से काम कर रहा हूँ; मुझे कोई अंदाज़ा नहीं है कि मैं क्या कर रहा हूँ।",
    },
    {
        "en": "She's a social butterfly; always flitting from party to party.",
        "hi": "वह एक सामाजिक तितली है; हमेशा पार्टी से पार्टी में उड़ती रहती है।",
    },
    {
        "en": "I'm feeling a bit spaced out today; I didn't get much sleep.",
        "hi": "आज मैं थोड़ा खोया हुआ महसूस कर रहा हूँ; मुझे ज़्यादा नींद नहीं आई।",
    },
    {
        "en": "Let's take a walk down memory lane; remember the good old days?",
        "hi": "चलो पुरानी यादों में खो जाते हैं; पुराने अच्छे दिनों को याद करते हैं।",
    },
    {
        "en": "That's a real Catch-22 situation; no matter what you do, you're stuck.",
        "hi": "यह सच में एक कैच-22 स्थिति है; आप कुछ भी करें, आप फंसे हुए हैं।",
    },
    {
        "en": "He's always trying to stir the pot; he loves to cause trouble.",
        "hi": "वह हमेशा आग लगाने की कोशिश करता रहता है; उसे परेशानी पैदा करना पसंद है।",
    },
    {
        "en": "I'm feeling a bit under the weather; I think I'm coming down with something.",
        "hi": "मैं थोड़ा बीमार महसूस कर रहा हूँ; मुझे लगता है कि मैं कुछ पकड़ रहा हूँ।",
    },
    {
        "en": "Let's get the ball rolling; we need to start this project.",
        "hi": "चलो काम शुरू करते हैं; हमें इस प्रोजेक्ट को शुरू करने की ज़रूरत है।",
    },
    {
        "en": "That's a real shot in the dark; we have no idea if it will work.",
        "hi": "यह सच में अंधेरे में तीर चलाने जैसा है; हमें कोई अंदाज़ा नहीं है कि यह काम करेगा या नहीं।",
    },
    {
        "en": "He's a bit of a lone wolf; he prefers to work alone.",
        "hi": "वह थोड़ा अकेला रहने वाला है; वह अकेले काम करना पसंद करता है।",
    },
    {
        "en": "Let's play it by ear; we'll see what happens and adjust accordingly.",
        "hi": "चलो देखते हैं आगे क्या होता है; हम देखेंगे कि क्या होता है और उसके अनुसार काम करेंगे।",
    },
    {
        "en": "That's a tough pill to swallow; it's hard to accept.",
        "hi": "यह स्वीकार करना मुश्किल है; इसे स्वीकार करना मुश्किल है।",
    },
    {
        "en": "Don't jump the gun; let's wait for all the information.",
        "hi": "जल्दबाजी मत करो; चलो सारी जानकारी का इंतजार करते हैं।",
    },
    {
        "en": "I'm feeling a bit off today; I think I need some rest.",
        "hi": "आज मेरी तबीयत थोड़ी ठीक नहीं है; मुझे लगता है मुझे आराम की ज़रूरत है।",
    },
    {
        "en": "That's a wake-up call; we need to take this seriously.",
        "hi": "यह एक चेतावनी है; हमें इसे गंभीरता से लेने की ज़रूरत है।",
    },
    {
        "en": "It's a win-win situation; everyone benefits.",
        "hi": "यह दोनों पक्षों के लिए फायदेमंद है; हर किसी को फायदा होता है।",
    },
    {
        "en": "He's a chatterbox; he never stops talking.",
        "hi": "वह बहुत बातूनी है; वह कभी बात करना बंद नहीं करता।",
    },
    {
        "en": "I'm feeling a bit drained; I need to recharge.",
        "hi": "मैं थोड़ा थका हुआ महसूस कर रहा हूँ; मुझे रिचार्ज करने की ज़रूरत है।",
    },
    {
        "en": "Let's table this for now; we can come back to it later.",
        "hi": "चलो इसे अभी के लिए रोक देते हैं; हम बाद में इस पर वापस आ सकते हैं।",
    },
    {
        "en": "That's a can of worms; it's a complicated problem.",
        "hi": "यह एक जटिल समस्या है; यह एक जटिल समस्या है।",
    },
    {
        "en": "Don't rock the boat; let's not cause any trouble.",
        "hi": "हालात मत बिगाड़ो; चलो कोई परेशानी नहीं करते।",
    },
    {
        "en": "He's a go-getter; he's always striving for success.",
        "hi": "वह मेहनती है; वह हमेशा सफलता के लिए प्रयास करता रहता है।",
    },
    {
        "en": "I'm feeling a bit under pressure; I need to relax.",
        "hi": "मैं थोड़ा दबाव महसूस कर रहा हूँ; मुझे आराम करने की ज़रूरत है।",
    },
    {
        "en": "Let's wrap things up; we're running out of time.",
        "hi": "चलो सब कुछ समेटते हैं; हमारे पास समय कम है।",
    },
    {
        "en": "That's a ballpark figure; it's just an estimate.",
        "hi": "यह एक अनुमानित आंकड़ा है; यह सिर्फ एक अनुमान है।",
    },
    {
        "en": "It's a learning curve; it takes time to master.",
        "hi": "यह सीखने की प्रक्रिया है; इसे महारत हासिल करने में समय लगता है।",
    },
    {
        "en": "He's a mover and shaker; he's an influential person.",
        "hi": "वह प्रभावशाली व्यक्ति है; वह एक प्रभावशाली व्यक्ति है।",
    },
    {
        "en": "I'm feeling a bit overwhelmed; I need to take a break.",
        "hi": "मैं थोड़ा अभिभूत महसूस कर रहा हूँ; मुझे ब्रेक लेने की ज़रूरत है।",
    },
    {
        "en": "Let's circle back to this later; we can discuss it further.",
        "hi": "चलो बाद में इस पर फिर से बात करते हैं; हम इस पर और चर्चा कर सकते हैं।",
    },
    {
        "en": "That's a game changer; it will revolutionize everything.",
        "hi": "यह सब कुछ बदल देगा; यह सब कुछ बदल देगा।",
    },
    {
        "en": "Don't bite off more than you can chew; be realistic about your abilities.",
        "hi": "अपनी क्षमता से ज़्यादा काम मत लो; अपनी क्षमताओं के बारे में यथार्थवादी बनें।",
    },
    {
        "en": "He's an early bird; he's always up before dawn.",
        "hi": "वह सुबह जल्दी उठने वाला है; वह हमेशा भोर से पहले उठता है।",
    },
    {
        "en": "I'm feeling a bit restless; I need to do something.",
        "hi": "मैं थोड़ा बेचैन महसूस कर रहा हूँ; मुझे कुछ करने की ज़रूरत है।",
    },
    {
        "en": "Let's brainstorm some ideas; we need to find a solution.",
        "hi": "चलो कुछ विचार मंथन करते हैं; हमें एक समाधान खोजने की ज़रूरत है।",
    },
    {
        "en": "That's a red flag; we need to be cautious.",
        "hi": "यह खतरे का संकेत है; हमें सावधान रहने की ज़रूरत है।",
    },
    {
        "en": "It's a tough crowd; they're hard to please.",
        "hi": "यह मुश्किल लोगों का समूह है; उन्हें खुश करना मुश्किल है।",
    },
    {
        "en": "He's a smooth talker; he can convince anyone of anything.",
        "hi": "वह मीठी-मीठी बातें करने वाला है; वह किसी को भी किसी भी चीज़ के लिए मना सकता है।",
    },
    {
        "en": "I'm feeling a bit wired; I have too much energy.",
        "hi": "मैं थोड़ा उत्तेजित महसूस कर रहा हूँ; मेरे पास बहुत ज़्यादा ऊर्जा है।",
    },
    {
        "en": "Let's get together sometime; we need to catch up.",
        "hi": "चलो कभी मिलते हैं; हमें बातचीत करनी है।",
    },
    {
        "en": "That's a golden opportunity; we can't afford to miss it.",
        "hi": "यह एक सुनहरा अवसर है; हम इसे खोने का जोखिम नहीं उठा सकते।",
    },
    {
        "en": "Don't rain on my parade; let me enjoy this moment.",
        "hi": "मेरा मज़ा किरकिरा मत करो; मुझे इस पल का आनंद लेने दो।",
    },
    {
        "en": "I'm feeling a bit foggy; I can't think straight.",
        "hi": "मेरा दिमाग थोड़ा धुंधला है; मैं सीधे नहीं सोच सकता।",
    },
    {
        "en": "Let's think outside the box; we need to find a creative solution.",
        "hi": "चलो कुछ अलग तरह से सोचते हैं; हमें एक रचनात्मक समाधान खोजने की ज़रूरत है।",
    },
    {
        "en": "It's a long haul; it will take a lot of time and effort.",
        "hi": "यह एक लंबी प्रक्रिया है; इसमें बहुत समय और प्रयास लगेगा।",
    },
    {
        "en": "I'm feeling a bit zonked; I'm completely exhausted.",
        "hi": "मैं बहुत थका हुआ महसूस कर रहा हूँ; मैं पूरी तरह से थका हुआ हूँ।",
    },
    {
        "en": "Let's play devil's advocate; we need to consider all sides of the issue.",
        "hi": "चलो विपरीत दृष्टिकोण से बहस करते हैं; हमें मुद्दे के सभी पहलुओं पर विचार करने की ज़रूरत है।",
    },
    {
        "en": "That's a silver lining; there's still some hope.",
        "hi": "यह मुश्किल में भी एक अच्छी बात है; अभी भी कुछ उम्मीद है।",
    },
    {
        "en": "He's a breath of fresh air; he's a welcome change.",
        "hi": "वह ताजगी का एहसास कराता है; वह एक स्वागत योग्य बदलाव है।",
    },
    {
        "en": "I'm feeling a bit jittery; I'm nervous and anxious.",
        "hi": "मैं थोड़ा घबराया हुआ महसूस कर रहा हूँ; मैं घबराया हुआ और चिंतित हूँ।",
    },
    {
        "en": "Let's get on the same page; we need to be in agreement.",
        "hi": "चलो एक ही बात पर सहमत होते हैं; हमें सहमत होने की ज़रूरत है।",
    },
    {
        "en": "It's a vicious cycle; it's hard to break free.",
        "hi": "यह एक दुष्चक्र है; इससे बाहर निकलना मुश्किल है।",
    },
    {
        "en": "He's a man of few words; he doesn't talk much.",
        "hi": "वह कम बोलने वाला है; वह ज़्यादा बात नहीं करता।",
    },
    {
        "en": "Let's cut to the chase; what's the main point?",
        "hi": "सीधे मुद्दे पर आते हैं; मुख्य बात क्या है?",
    },
    {
        "en": "That's a recipe for disaster; it's bound to go wrong.",
        "hi": "यह बर्बादी का कारण बनेगा; यह गलत होने के लिए बाध्य है।",
    },
    {
        "en": "Don't beat yourself up about it; it's not your fault.",
        "hi": "इसके बारे में खुद को दोष मत दो; यह तुम्हारी गलती नहीं है।",
    },
    {
        "en": "He's a people person; he's good with people.",
        "hi": "वह लोगों से आसानी से घुलमिल जाता है; वह लोगों के साथ अच्छा है।",
    },
    {
        "en": "Let's think big; we need to be ambitious.",
        "hi": "चलो बड़ा सोचते हैं; हमें महत्वाकांक्षी होने की ज़रूरत है।",
    },
    {
        "en": "That's a stepping stone; it's a step in the right direction.",
        "hi": "यह एक शुरुआती कदम है; यह सही दिशा में एक कदम है।",
    },
    {
        "en": "It's a whole new ballgame; things are completely different now.",
        "hi": "यह बिल्कुल नई स्थिति है; अब चीजें पूरी तरह से अलग हैं।",
    },
    {
        "en": "He's a trendsetter; he's always ahead of the curve.",
        "hi": "वह नए चलन शुरू करने वाला है; वह हमेशा सबसे आगे रहता है।",
    },
    {
        "en": "I'm feeling a bit out of my depth; I'm not sure I can handle this.",
        "hi": "मुझे यह थोड़ा मुश्किल लग रहा है; मुझे यकीन नहीं है कि मैं इसे संभाल सकता हूँ।",
    },
    {
        "en": "Let's keep it simple; we don't need to overcomplicate things.",
        "hi": "चलो इसे सरल रखते हैं; हमें चीजों को ज़्यादा जटिल बनाने की ज़रूरत नहीं है।",
    },
    {
        "en": "Don't get your hopes up too high; be prepared for disappointment.",
        "hi": "ज़्यादा उम्मीदें मत रखो; निराशा के लिए तैयार रहो।",
    },
    {
        "en": "He's a diamond in the rough; he has potential but needs to be polished.",
        "hi": "उसमें प्रतिभा है, पर उसे तराशने की ज़रूरत है; उसमें क्षमता है लेकिन उसे चमकाने की ज़रूरत है।",
    },
    {
        "en": "Let's go with the flow; we'll see what happens.",
        "hi": "चलो जैसे चल रहा है, चलने देते हैं; हम देखेंगे कि क्या होता है।",
    },
    {
        "en": "That's a real bummer; it's really disappointing.",
        "hi": "यह सच में निराशाजनक है; यह सच में निराशाजनक है।",
    },
    {
        "en": "The whole nine yards; everything, completely.",
        "hi": "पूरी तरह से; सब कुछ, पूरी तरह से।",
    },
    {
        "en": "Bite off more than you can chew; take on more than you can handle.",
        "hi": "अपनी क्षमता से ज़्यादा काम लेना; जितना तुम संभाल सकते हो उससे ज़्यादा काम लेना।",
    },
    {
        "en": "Go the extra mile; put in extra effort.",
        "hi": "अतिरिक्त प्रयास करना; अतिरिक्त प्रयास करना।",
    },
    {
        "en": "Hit the nail on the head; say exactly the right thing.",
        "hi": "सही बात कहना; बिल्कुल सही बात कहना।",
    },
    {
        "en": "Jump on the bandwagon; follow a trend.",
        "hi": "देखा-देखी करना; एक चलन का पालन करना।",
    },
    {
        "en": "Leave no stone unturned; try everything possible.",
        "hi": "हर संभव प्रयास करना; हर संभव कोशिश करना।",
    },
    {
        "en": "Make a long story short; summarize.",
        "hi": "संक्षेप में कहना; संक्षेप में बताना।",
    },
    { "en": "On the same page; in agreement.", "hi": "एक ही राय होना; सहमत होना।" },
    {
        "en": "Play it by ear; improvise.",
        "hi": "बिना तैयारी के काम करना; तात्कालिक रूप से करना।",
    },
    { "en": "Take a rain check; postpone.", "hi": "फिर कभी करना; स्थगित करना।" },
    {
        "en": "He's pulling my leg, isn't he?; He's joking, right?",
        "hi": "वह मेरी टांग खींच रहा है, है ना?; वह मजाक कर रहा है, है ना?",
    },
    {
        "en": "I'm feeling a bit out of sync with everyone today; I don't fit in.",
        "hi": "आज मैं थोड़ा सबसे अलग महसूस कर रहा हूँ; मैं फिट नहीं बैठता।",
    },
    {
        "en": "Let's not beat around the bush and get straight to the point; let's be direct.",
        "hi": "चलो गोलमोल बातें नहीं करते और सीधे मुद्दे पर आते हैं; चलो सीधे बात करते हैं।",
    },
    {
        "en": "That situation was a real Catch-22; whatever we did, we'd be in trouble; there was no good solution.",
        "hi": "वह स्थिति सच में एक कैच-22 थी; हम कुछ भी करते, मुसीबत में पड़ते; कोई अच्छा समाधान नहीं था।",
    },
    {
        "en": "She's got a bee in her bonnet about the new regulations; she's obsessed with them.",
        "hi": "नई नियमों को लेकर उसके दिमाग में कुछ परेशानी चल रही है; वह उनसे ग्रस्त है।",
    },
    {
        "en": "He's always trying to cut corners, which often leads to more problems down the line; he's taking shortcuts.",
        "hi": "वह हमेशा शॉर्टकट मारने की कोशिश करता है, जिससे अक्सर आगे चलकर और ज़्यादा समस्याएँ आती हैं; वह शॉर्टकट ले रहा है।",
    },
    {
        "en": "If you don't get your ducks in a row, this project is going to fall apart; you need to organize things.",
        "hi": "अगर तुम सब कुछ ठीक से व्यवस्थित नहीं करोगे, तो यह प्रोजेक्ट बिगड़ जाएगा; तुम्हें चीजों को व्यवस्थित करने की ज़रूरत है।",
    },
    {
        "en": "Despite the initial setbacks, they managed to turn the tables and come out on top; they overcame the challenges.",
        "hi": "शुरुआती झटकों के बावजूद, वे स्थिति पलटने और शीर्ष पर आने में कामयाब रहे; उन्होंने चुनौतियों पर काबू पाया।",
    },
    {
        "en": "I'm walking on eggshells around him because he's been so touchy lately; I'm being very careful.",
        "hi": "मैं उसके आसपास बहुत सावधानी से पेश आ रहा हूँ क्योंकि वह आजकल बहुत चिड़चिड़ा है; मैं बहुत सावधान रह रहा हूँ।",
    },
    {
        "en": "Let's just agree to disagree and move on to a different topic; we're not going to change each other's minds.",
        "hi": "चलो बस इस बात पर असहमत होते हैं और किसी और विषय पर चलते हैं; हम एक-दूसरे के विचारों को बदलने वाले नहीं हैं।",
    },
    {
        "en": "He's got a silver tongue; he could sell ice to Eskimos.",
        "hi": "उसकी ज़ुबान में शहद है; वो तो बर्फ को एस्किमो को भी बेच सकता है।",
    },
    {
        "en": "That's the way the cookie crumbles.",
        "hi": "दुनिया ऐसी ही है। / यही जीवन है।",
    },
    {
        "en": "I'm feeling a bit off today, like I'm running on fumes.",
        "hi": "आज मैं थोड़ा अजीब महसूस कर रहा हूँ, जैसे बस घिसट रहा हूँ।",
    },
    {
        "en": "Let's take a rain check on that; I'm swamped right now.",
        "hi": "चलो उसे फिर कभी करते हैं; मैं अभी बहुत व्यस्त हूँ।",
    },
    {
        "en": "He's a diamond in the rough, but he's got potential.",
        "hi": "वो है तो अनगढ़ हीरा, पर उसमें चमक है।",
    },
    {
        "en": "Don't get your knickers in a twist; it's not a big deal.",
        "hi": "इतना परेशान मत हो; कोई बड़ी बात नहीं है।",
    },
    {
        "en": "She's got a heart of gold; she'd do anything for you.",
        "hi": "उसका दिल सोने का है; वो तुम्हारे लिए कुछ भी करेगी।",
    },
    {
        "en": "He's a bit of a rolling stone; never settles down.",
        "hi": "वो थोड़ा घुमक्कड़ है; कभी टिकता नहीं है।",
    },
    {
        "en": "Let's get the ball rolling on this project; time is of the essence.",
        "hi": "चलो इस प्रोजेक्ट पर काम शुरू करते हैं; समय बहुत कीमती है।",
    },
    {
        "en": "That's a real shot in the dark, but we've got nothing to lose.",
        "hi": "ये तो अंधेरे में तीर चलाने जैसा है, पर हमारे पास खोने को कुछ नहीं है।",
    },
    {
        "en": "He's a bit of a lone wolf; prefers his own company.",
        "hi": "वो थोड़ा अकेला रहने वाला है; अपनी संगति पसंद करता है।",
    },
    {
        "en": "I'm feeling a bit out of sorts; can't quite put my finger on it.",
        "hi": "आज मैं थोड़ा अजीब महसूस कर रहा हूँ; समझ नहीं आ रहा क्या है।",
    },
    {
        "en": "Let's play it by ear and see how things pan out.",
        "hi": "चलो देखते हैं क्या होता है और उसके हिसाब से चलते हैं।",
    },
    {
        "en": "That's a tough pill to swallow, but we have to face reality.",
        "hi": "ये मानना मुश्किल है, पर हमें सच्चाई का सामना करना होगा।",
    },
    {
        "en": "Don't jump the gun; let's wait for all the facts to emerge.",
        "hi": "जल्दबाज़ी मत करो; चलो सारे तथ्य सामने आने का इंतज़ार करते हैं।",
    },
    {
        "en": "He's a walking encyclopedia; knows something about everything.",
        "hi": "वो तो चलता-फिरता विश्वकोश है; उसे हर चीज़ के बारे में कुछ न कुछ पता है।",
    },
    {
        "en": "I'm feeling a bit off today; I think I need some R&R.",
        "hi": "आज मैं थोड़ा अजीब महसूस कर रहा हूँ; लगता है मुझे आराम की ज़रूरत है।",
    },
    {
        "en": "Let's touch base next week and see where we're at.",
        "hi": "चलो अगले हफ्ते बात करते हैं और देखते हैं हम कहाँ तक पहुँचे हैं।",
    },
    {
        "en": "That's a wake-up call; we need to change our ways.",
        "hi": "ये एक चेतावनी है; हमें अपने तरीके बदलने होंगे।",
    },
    {
        "en": "It's a win-win situation; everyone comes out ahead.",
        "hi": "ये तो फायदे का सौदा है; हर कोई आगे बढ़ेगा।",
    },
    {
        "en": "He's a chatterbox; never shuts up.",
        "hi": "वो तो बातूनी है; कभी चुप नहीं होता।",
    },
    {
        "en": "I'm feeling a bit drained; I need to recharge my batteries.",
        "hi": "मैं थोड़ा थका हुआ महसूस कर रहा हूँ; मुझे अपनी बैटरी चार्ज करने की ज़रूरत है।",
    },
    {
        "en": "Let's table this for now and revisit it later.",
        "hi": "चलो इसे अभी रोक देते हैं और बाद में फिर से देखते हैं।",
    },
    {
        "en": "That's a can of worms; better left unopened.",
        "hi": "ये तो झंझट का पिटारा है; बेहतर है इसे न खोलें।",
    },
    {
        "en": "Don't rock the boat; let's keep things as they are.",
        "hi": "हालात मत बिगाड़ो; चलो चीज़ों को जैसा है वैसा ही रहने देते हैं।",
    },
    {
        "en": "He's a go-getter; always striving for the top.",
        "hi": "वो मेहनती है; हमेशा ऊपर जाने की कोशिश करता है।",
    },
    {
        "en": "I'm feeling a bit under pressure; I need to take a breather.",
        "hi": "मैं थोड़ा दबाव महसूस कर रहा हूँ; मुझे सांस लेने की ज़रूरत है।",
    },
    {
        "en": "Let's wrap things up; it's getting late.",
        "hi": "चलो सब कुछ समेटते हैं; देर हो रही है।",
    },
    {
        "en": "That's a ballpark figure; it's just a rough estimate.",
        "hi": "ये तो अंदाज़ा है; बस एक मोटा-मोटा अनुमान।",
    },
    {
        "en": "It's a learning curve; takes time to get the hang of it.",
        "hi": "ये सीखने की प्रक्रिया है; इसमें समय लगता है।",
    },
    {
        "en": "He's a mover and shaker; knows how to get things done.",
        "hi": "वो प्रभावशाली व्यक्ति है; जानता है काम कैसे निकालना है।",
    },
    {
        "en": "I'm feeling a bit overwhelmed; I need to step back for a moment.",
        "hi": "मैं थोड़ा अभिभूत महसूस कर रहा हूँ; मुझे एक पल के लिए पीछे हटने की ज़रूरत है।",
    },
    {
        "en": "Let's circle back to this next week and see where we stand.",
        "hi": "चलो अगले हफ्ते इस पर फिर से बात करते हैं और देखते हैं हम कहाँ खड़े हैं।",
    },
    {
        "en": "That's a game changer; it'll revolutionize the industry.",
        "hi": "ये तो सब कुछ बदल देगा; उद्योग में क्रांति ला देगा।",
    },
    {
        "en": "Don't bite off more than you can chew; be realistic about your limitations.",
        "hi": "अपनी क्षमता से ज़्यादा काम मत लो; अपनी सीमाओं के बारे में यथार्थवादी बनो।",
    },
    {
        "en": "He's an early bird; always up with the lark.",
        "hi": "वो सुबह जल्दी उठने वाला है; हमेशा सूरज निकलने से पहले उठता है।",
    },
    {
        "en": "I'm feeling a bit restless; I need to get out and do something.",
        "hi": "मैं थोड़ा बेचैन महसूस कर रहा हूँ; मुझे बाहर निकलकर कुछ करने की ज़रूरत है।",
    },
    {
        "en": "Let's brainstorm some ideas; we need to come up with a solution.",
        "hi": "चलो कुछ विचार मंथन करते हैं; हमें एक समाधान निकालने की ज़रूरत है।",
    },
    {
        "en": "That's a red flag; we need to proceed with caution.",
        "hi": "ये खतरे का संकेत है; हमें सावधानी से आगे बढ़ने की ज़रूरत है।",
    },
    {
        "en": "He's a smooth talker; can charm the birds out of the trees.",
        "hi": "वो मीठी-मीठी बातें करने वाला है; पेड़ों से पक्षियों को भी मोहित कर सकता है।",
    },
    {
        "en": "I'm feeling a bit wired; I've had too much coffee.",
        "hi": "मैं थोड़ा उत्तेजित महसूस कर रहा हूँ; मैंने बहुत ज़्यादा कॉफ़ी पी ली है।",
    },
    {
        "en": "Let's get together sometime; we need to catch up over coffee.",
        "hi": "चलो कभी मिलते हैं; हमें कॉफ़ी पर बातचीत करनी है।",
    },
    {
        "en": "That's a golden opportunity; we can't afford to let it pass us by.",
        "hi": "ये एक सुनहरा अवसर है; हम इसे हाथ से जाने नहीं दे सकते।",
    },
    {
        "en": "Don't rain on my parade; let me have my moment in the sun.",
        "hi": "मेरा मज़ा किरकिरा मत करो; मुझे अपनी सफलता का आनंद लेने दो।",
    },
    {
        "en": "He's a workaholic; lives and breathes his job.",
        "hi": "वो काम के प्रति बहुत समर्पित है; जीता और मरता है अपने काम के लिए।",
    },
    {
        "en": "I'm feeling a bit foggy; I can't seem to concentrate.",
        "hi": "मेरा दिमाग थोड़ा धुंधला है; मैं ध्यान केंद्रित नहीं कर पा रहा हूँ।",
    },
    {
        "en": "Let's think outside the box; we need a creative approach.",
        "hi": "चलो कुछ अलग तरह से सोचते हैं; हमें एक रचनात्मक दृष्टिकोण की ज़रूरत है।",
    },
    {
        "en": "That's a grey area; it's open to interpretation.",
        "hi": "ये अस्पष्ट क्षेत्र है; इसकी व्याख्या की जा सकती है।",
    },
    {
        "en": "It's a long haul; it's going to take a lot of perseverance.",
        "hi": "ये एक लंबी प्रक्रिया है; इसमें बहुत धैर्य लगेगा।",
    },
    {
        "en": "He's a team player; always willing to lend a hand.",
        "hi": "वो टीम में मिलकर काम करने वाला है; हमेशा मदद करने को तैयार रहता है।",
    },
    {
        "en": "I'm feeling a bit zonked; I'm ready for bed.",
        "hi": "मैं बहुत थका हुआ महसूस कर रहा हूँ; मैं सोने के लिए तैयार हूँ।",
    },
    {
        "en": "Let's play devil's advocate; let's look at it from a different angle.",
        "hi": "चलो विपरीत दृष्टिकोण से बहस करते हैं; चलो इसे एक अलग कोण से देखते हैं।",
    },
    {
        "en": "That's a silver lining; there's always a glimmer of hope.",
        "hi": "ये मुश्किल में भी एक अच्छी बात है; हमेशा उम्मीद की किरण होती है।",
    },
    {
        "en": "He's a breath of fresh air; a welcome change from the norm.",
        "hi": "वो ताजगी का एहसास कराता है; सामान्य से एक स्वागत योग्य बदलाव है।",
    },
    {
        "en": "I'm feeling a bit jittery; I'm on edge about the presentation.",
        "hi": "मैं थोड़ा घबराया हुआ महसूस कर रहा हूँ; मैं प्रस्तुति को लेकर चिंतित हूँ।",
    },
    {
        "en": "Let's get on the same page; we need to be in sync.",
        "hi": "चलो एक ही बात पर सहमत होते हैं; हमें तालमेल बिठाने की ज़रूरत है।",
    },
    {
        "en": "That's a tough break; it's a real setback.",
        "hi": "ये बहुत बुरा हुआ; ये एक वास्तविक झटका है।",
    },
    {
        "en": "It's a vicious cycle; it's hard to escape.",
        "hi": "ये एक दुष्चक्र है; इससे बचना मुश्किल है।",
    },
    {
        "en": "He's a man of few words; lets his actions speak for themselves.",
        "hi": "वो कम बोलने वाला है; उसके काम खुद बोलते हैं।",
    },
    {
        "en": "Let's cut to the chase; what's the bottom line?",
        "hi": "सीधे मुद्दे पर आते हैं; आखिर बात क्या है?",
    },
    {
        "en": "That's a recipe for disaster; it's a sure-fire way to fail.",
        "hi": "ये बर्बादी का कारण बनेगा; ये असफल होने का एक निश्चित तरीका है।",
    },
    {
        "en": "Don't beat yourself up about it; everyone makes mistakes.",
        "hi": "इसके बारे में खुद को दोष मत दो; हर कोई गलती करता है।",
    },
    {
        "en": "He's a people person; gets along with everyone he meets.",
        "hi": "वो लोगों से आसानी से घुलमिल जाता है; वो हर उस व्यक्ति के साथ अच्छी तरह से मिलता है जिससे वो मिलता है।",
    },
    {
        "en": "Let's think big; let's aim for the stars.",
        "hi": "चलो बड़ा सोचते हैं; चलो सितारों के लिए लक्ष्य रखते हैं।",
    },
    {
        "en": "That's a stepping stone; it's a move in the right direction.",
        "hi": "ये एक शुरुआती कदम है; ये सही दिशा में एक कदम है।",
    },
    {
        "en": "He's a trendsetter; always ahead of the curve.",
        "hi": "वो नए चलन शुरू करने वाला है; हमेशा सबसे आगे रहता है।",
    },
    {
        "en": "I'm feeling a bit out of my depth; I'm not sure I'm up to the task.",
        "hi": "मुझे यह थोड़ा मुश्किल लग रहा है; मुझे यकीन नहीं है कि मैं इस काम को कर पाऊँगा।",
    },
    {
        "en": "Let's keep it simple; don't overcomplicate things.",
        "hi": "चलो इसे सरल रखते हैं; चीजों को ज़्यादा जटिल मत बनाओ।",
    },
    {
        "en": "Don't get your hopes up too high; be prepared for disappointment.",
        "hi": "ज़्यादा उम्मीदें मत रखो; निराशा के लिए तैयार रहो।",
    },
    {
        "en": "He's a diamond in the rough; needs some polishing but has great potential.",
        "hi": "उसमें प्रतिभा है, पर उसे तराशने की ज़रूरत है; उसे चमकाने की ज़रूरत है लेकिन उसमें बहुत क्षमता है।",
    },
    {
        "en": "Let's go with the flow; see what happens and adapt accordingly.",
        "hi": "चलो जैसे चल रहा है, चलने देते हैं; देखते हैं क्या होता है और उसके अनुसार ढलते हैं।",
    },
    {
        "en": "That's a real bummer; it's a real letdown.",
        "hi": "यह सच में निराशाजनक है; यह सच में निराशाजनक है।",
    },
    {
        "en": "The whole nine yards; everything, from start to finish.",
        "hi": "पूरी तरह से; सब कुछ, शुरू से अंत तक।",
    },
    {
        "en": "Bite off more than you can chew; take on more than you can handle.",
        "hi": "अपनी क्षमता से ज़्यादा काम लेना; जितना तुम संभाल सकते हो उससे ज़्यादा काम लेना।",
    },
    {
        "en": "Go the extra mile; put in that extra bit of effort.",
        "hi": "अतिरिक्त प्रयास करना; अतिरिक्त प्रयास करना।",
    },
    {
        "en": "Hit the nail on the head; say exactly the right thing.",
        "hi": "सही बात कहना; बिल्कुल सही बात कहना।",
    },
    {
        "en": "Jump on the bandwagon; follow the latest trend.",
        "hi": "देखा-देखी करना; नवीनतम चलन का पालन करना।",
    },
    {
        "en": "Leave no stone unturned; try every possible avenue.",
        "hi": "हर संभव प्रयास करना; हर संभव कोशिश करना।",
    },
    {
        "en": "Make a long story short; get to the point.",
        "hi": "संक्षेप में कहना; मुद्दे पर आना।",
    },
    {
        "en": "On the same page; in agreement with each other.",
        "hi": "एक ही राय होना; एक दूसरे से सहमत होना।",
    },
    {
        "en": "Play it by ear; see how things go and adjust accordingly.",
        "hi": "बिना तैयारी के काम करना; देखें कि चीजें कैसे होती हैं और उसके अनुसार समायोजित करें।",
    },
    {
        "en": "Take a rain check; postpone to another time.",
        "hi": "फिर कभी करना; किसी और समय के लिए स्थगित करना।",
    },
    {
        "en": "He's pulling my leg, isn't he?; He's just teasing me, right?",
        "hi": "वह मेरी टांग खींच रहा है, है ना?; वह सिर्फ मुझे चिढ़ा रहा है, है ना?",
    },
    {
        "en": "I'm feeling a bit out of sync with everyone today; I just don't connect.",
        "hi": "आज मैं थोड़ा सबसे अलग महसूस कर रहा हूँ; मैं बस जुड़ नहीं पाता।",
    },
    {
        "en": "Let's not beat around the bush and get straight to the point; let's be direct and honest.",
        "hi": "चलो गोलमोल बातें नहीं करते और सीधे मुद्दे पर आते हैं; चलो सीधे और ईमानदार रहें।",
    },
    {
        "en": "That situation was a real Catch-22; whatever we did, we were doomed; there was no good option.",
        "hi": "वह स्थिति सच में एक कैच-22 थी; हम कुछ भी करते, हम बर्बाद हो जाते; कोई अच्छा विकल्प नहीं था।",
    },
    {
        "en": "She's got a bee in her bonnet about the new regulations; she's obsessed and won't stop talking about them.",
        "hi": "नई नियमों को लेकर उसके दिमाग में कुछ परेशानी चल रही है; वह ग्रस्त है और उनके बारे में बात करना बंद नहीं करेगी।",
    },
    {
        "en": "He's always trying to cut corners, which often leads to more problems down the line; he's taking risky shortcuts.",
        "hi": "वह हमेशा शॉर्टकट मारने की कोशिश करता है, जिससे अक्सर आगे चलकर और ज़्यादा समस्याएँ आती हैं; वह जोखिम भरे शॉर्टकट ले रहा है।",
    },
    {
        "en": "If you don't get your ducks in a row, this project is going to fail; you need to organize and plan properly.",
        "hi": "अगर तुम सब कुछ ठीक से व्यवस्थित नहीं करोगे, तो यह प्रोजेक्ट बिगड़ जाएगा; तुम्हें ठीक से व्यवस्थित और योजना बनाने की ज़रूरत है।",
    },
    {
        "en": "Despite the initial setbacks, they managed to turn the tables and come out on top; they overcame adversity and succeeded.",
        "hi": "शुरुआती झटकों के बावजूद, वे स्थिति पलटने और शीर्ष पर आने में कामयाब रहे; उन्होंने विपरीत परिस्थितियों पर काबू पाया और सफल हुए।",
    },
    {
        "en": "I'm walking on eggshells around him because he's been so touchy lately; I have to be very careful what I say.",
        "hi": "मैं उसके आसपास बहुत सावधानी से पेश आ रहा हूँ क्योंकि वह आजकल बहुत चिड़चिड़ा है; मुझे बहुत सावधान रहना होगा कि मैं क्या कहता हूँ।",
    },
    {
        "en": "Let's just agree to disagree and move on to a different topic; we're never going to see eye to eye on this.",
        "hi": "चलो बस इस बात पर असहमत होते हैं और किसी और विषय पर चलते हैं; हम इस पर कभी भी सहमत नहीं होंगे।",
    },
    {
        "en": "He's got a heart of gold, always helping others.",
        "hi": "उसका दिल सोने का है, हमेशा दूसरों की मदद करता है।",
    },
    {
        "en": "That's a tough act to follow, he set the bar high.",
        "hi": "यह एक मुश्किल काम है, उसने स्तर ऊंचा कर दिया।",
    },
    {
        "en": "I'm feeling a bit under the weather, might stay in tonight.",
        "hi": "मैं थोड़ा बीमार महसूस कर रहा हूँ, शायद आज रात अंदर ही रहूँ।",
    },
    {
        "en": "Let's get the ball rolling, time to start planning.",
        "hi": "चलो काम शुरू करते हैं, योजना बनाने का समय है।",
    },
    {
        "en": "That's a shot in the dark, but worth a try anyway.",
        "hi": "यह अंधेरे में तीर चलाने जैसा है, लेकिन फिर भी कोशिश करने लायक है।",
    },
    {
        "en": "He's a lone wolf, prefers working independently.",
        "hi": "वह अकेला रहने वाला है, स्वतंत्र रूप से काम करना पसंद करता है।",
    },
    {
        "en": "I'm feeling out of sorts, can't explain what's wrong.",
        "hi": "मैं थोड़ा असहज महसूस कर रहा हूँ, बता नहीं सकता क्या गलत है।",
    },
    {
        "en": "Let's play it by ear, see what happens and adapt.",
        "hi": "चलो देखते हैं आगे क्या होता है, और उसके अनुसार ढलते हैं।",
    },
    {
        "en": "That's a tough pill to swallow, but we must accept it.",
        "hi": "यह स्वीकार करना मुश्किल है, लेकिन हमें इसे स्वीकार करना होगा।",
    },
    {
        "en": "Don't jump the gun, let's wait for all the details.",
        "hi": "जल्दबाजी मत करो, चलो सभी विवरणों का इंतजार करते हैं।",
    },
    {
        "en": "He's a walking encyclopedia, knows everything about history.",
        "hi": "वह चलता-फिरता विश्वकोश है, इतिहास के बारे में सब कुछ जानता है।",
    },
    {
        "en": "I'm feeling a bit off today, need some rest and relaxation.",
        "hi": "आज मेरी तबीयत थोड़ी ठीक नहीं है, मुझे आराम और विश्राम की ज़रूरत है।",
    },
    {
        "en": "Let's touch base next week, discuss progress and next steps.",
        "hi": "चलो अगले हफ्ते बात करते हैं, प्रगति और अगले कदमों पर चर्चा करते हैं।",
    },
    {
        "en": "That's a wake-up call, we need to take action now.",
        "hi": "यह एक चेतावनी है, हमें अब कार्रवाई करने की ज़रूरत है।",
    },
    {
        "en": "It's a win-win situation, both sides benefit equally.",
        "hi": "यह दोनों पक्षों के लिए फायदेमंद है, दोनों पक्षों को समान रूप से लाभ होता है।",
    },
    {
        "en": "He's a chatterbox, never stops talking.",
        "hi": "वह बहुत बातूनी है, कभी बात करना बंद नहीं करता।",
    },
    {
        "en": "I'm feeling drained, need to recharge my energy levels.",
        "hi": "मैं थका हुआ महसूस कर रहा हूँ, मुझे अपनी ऊर्जा के स्तर को फिर से भरने की ज़रूरत है।",
    },
    {
        "en": "Let's table this for now, revisit it at the next meeting.",
        "hi": "चलो इसे अभी के लिए रोक देते हैं, अगली बैठक में इस पर फिर से विचार करते हैं।",
    },
    {
        "en": "That's a can of worms, better not to get involved.",
        "hi": "यह झंझट का पिटारा है, बेहतर है कि इसमें शामिल न हों।",
    },
    {
        "en": "Don't rock the boat, let's maintain the status quo.",
        "hi": "हालात मत बिगाड़ो, चलो यथास्थिति बनाए रखते हैं।",
    },
    {
        "en": "He's a go-getter, always striving for success.",
        "hi": "वह मेहनती है, हमेशा सफलता के लिए प्रयास करता रहता है।",
    },
    {
        "en": "I'm feeling under pressure, need to find a way to de-stress.",
        "hi": "मैं दबाव महसूस कर रहा हूँ, मुझे तनाव कम करने का एक तरीका खोजने की ज़रूरत है।",
    },
    {
        "en": "Let's wrap things up, time to head home for the day.",
        "hi": "चलो सब कुछ समेटते हैं, आज के लिए घर जाने का समय है।",
    },
    {
        "en": "That's a ballpark figure, just a rough estimate.",
        "hi": "यह एक अनुमानित आंकड़ा है, सिर्फ एक मोटा अनुमान।",
    },
    {
        "en": "It's a learning curve, takes time to acquire new skills.",
        "hi": "यह सीखने की प्रक्रिया है, नए कौशल हासिल करने में समय लगता है।",
    },
    {
        "en": "He's a mover and shaker, influential in the industry.",
        "hi": "वह प्रभावशाली व्यक्ति है, उद्योग में प्रभावशाली है।",
    },
    {
        "en": "I'm feeling overwhelmed, need to take a step back and breathe.",
        "hi": "मैं अभिभूत महसूस कर रहा हूँ, मुझे एक कदम पीछे हटने और सांस लेने की ज़रूरत है।",
    },
    {
        "en": "Let's circle back to this later, discuss it in more detail.",
        "hi": "चलो बाद में इस पर फिर से बात करते हैं, इस पर और विस्तार से चर्चा करते हैं।",
    },
    {
        "en": "That's a game changer, will revolutionize the market.",
        "hi": "यह सब कुछ बदल देगा, बाजार में क्रांति ला देगा।",
    },
    {
        "en": "Don't bite off more than you can chew, be realistic about your limits.",
        "hi": "अपनी क्षमता से ज़्यादा काम मत लो, अपनी सीमाओं के बारे में यथार्थवादी बनें।",
    },
    {
        "en": "He's an early bird, always up before sunrise.",
        "hi": "वह सुबह जल्दी उठने वाला है, हमेशा सूर्योदय से पहले उठता है।",
    },
    {
        "en": "I'm feeling restless, need to get out and do something active.",
        "hi": "मैं बेचैन महसूस कर रहा हूँ, मुझे बाहर निकलकर कुछ सक्रिय करने की ज़रूरत है।",
    },
    {
        "en": "Let's brainstorm some ideas, find a creative solution.",
        "hi": "चलो कुछ विचार मंथन करते हैं, एक रचनात्मक समाधान खोजते हैं।",
    },
    {
        "en": "That's a red flag, we need to be extra cautious.",
        "hi": "यह खतरे का संकेत है, हमें अतिरिक्त सावधान रहने की ज़रूरत है।",
    },
    {
        "en": "It's a tough crowd, difficult to impress.",
        "hi": "यह मुश्किल लोगों का समूह है, प्रभावित करना मुश्किल है।",
    },
    {
        "en": "I'm feeling wired, too much caffeine in my system.",
        "hi": "मैं उत्तेजित महसूस कर रहा हूँ, मेरे सिस्टम में बहुत ज़्यादा कैफीन है।",
    },
    {
        "en": "Let's get together sometime, catch up over dinner.",
        "hi": "चलो कभी मिलते हैं, रात के खाने पर बातचीत करते हैं।",
    },
    {
        "en": "That's a golden opportunity, we can't afford to miss it.",
        "hi": "यह एक सुनहरा अवसर है, हम इसे खोने का जोखिम नहीं उठा सकते।",
    },
    {
        "en": "Don't rain on my parade, let me enjoy my success.",
        "hi": "मेरा मज़ा किरकिरा मत करो, मुझे अपनी सफलता का आनंद लेने दो।",
    },
    {
        "en": "He's a workaholic, always putting in extra hours.",
        "hi": "वह काम के प्रति बहुत समर्पित है, हमेशा अतिरिक्त घंटे लगाता है।",
    },
    {
        "en": "I'm feeling foggy, can't seem to think clearly.",
        "hi": "मेरा दिमाग धुंधला है, मैं स्पष्ट रूप से नहीं सोच पा रहा हूँ।",
    },
    {
        "en": "Let's think outside the box, come up with a novel approach.",
        "hi": "चलो कुछ अलग तरह से सोचते हैं, एक उपन्यास दृष्टिकोण के साथ आते हैं।",
    },
    {
        "en": "That's a grey area, not clearly defined or regulated.",
        "hi": "यह अस्पष्ट क्षेत्र है, स्पष्ट रूप से परिभाषित या विनियमित नहीं है।",
    },
    {
        "en": "It's a long haul, requires patience and perseverance.",
        "hi": "यह एक लंबी प्रक्रिया है, धैर्य और दृढ़ता की आवश्यकता है।",
    },
    {
        "en": "He's a team player, always willing to collaborate.",
        "hi": "वह टीम में मिलकर काम करने वाला है, हमेशा सहयोग करने को तैयार रहता है।",
    },
    {
        "en": "I'm feeling zonked, ready to crash and go to sleep.",
        "hi": "मैं बहुत थका हुआ महसूस कर रहा हूँ, सोने के लिए तैयार हूँ।",
    },
    {
        "en": "Let's play devil's advocate, consider alternative viewpoints.",
        "hi": "चलो विपरीत दृष्टिकोण से बहस करते हैं, वैकल्पिक दृष्टिकोणों पर विचार करते हैं।",
    },
    {
        "en": "That's a silver lining, a positive aspect in a bad situation.",
        "hi": "यह मुश्किल में भी एक अच्छी बात है, बुरी स्थिति में एक सकारात्मक पहलू।",
    },
    {
        "en": "He's a breath of fresh air, brings new ideas and perspectives.",
        "hi": "वह ताजगी का एहसास कराता है, नए विचार और दृष्टिकोण लाता है।",
    },
    {
        "en": "I'm feeling jittery, nervous about the upcoming presentation.",
        "hi": "मैं घबराया हुआ महसूस कर रहा हूँ, आने वाली प्रस्तुति के बारे में चिंतित हूँ।",
    },
    {
        "en": "Let's get on the same page, ensure we all understand the plan.",
        "hi": "चलो एक ही बात पर सहमत होते हैं, सुनिश्चित करें कि हम सभी योजना को समझते हैं।",
    },
    {
        "en": "That's a tough break, a real stroke of bad luck.",
        "hi": "यह बहुत बुरा हुआ, बुरी किस्मत का एक वास्तविक झटका।",
    },
    {
        "en": "He's a man of few words, prefers to listen and observe.",
        "hi": "वह कम बोलने वाला है, सुनना और निरीक्षण करना पसंद करता है।",
    },
    {
        "en": "Let's cut to the chase, get straight to the point.",
        "hi": "सीधे मुद्दे पर आते हैं, सीधे मुद्दे पर आते हैं।",
    },
    {
        "en": "That's a recipe for disaster, likely to end badly.",
        "hi": "यह बर्बादी का कारण बनेगा, बुरी तरह से समाप्त होने की संभावना है।",
    },
    {
        "en": "Don't beat yourself up about it, everyone makes mistakes.",
        "hi": "इसके बारे में खुद को दोष मत दो, हर कोई गलती करता है।",
    },
    {
        "en": "He's a people person, enjoys interacting with others.",
        "hi": "वह लोगों से आसानी से घुलमिल जाता है, दूसरों के साथ बातचीत करने का आनंद लेता है।",
    },
    {
        "en": "Let's think big, aim for ambitious goals.",
        "hi": "चलो बड़ा सोचते हैं, महत्वाकांक्षी लक्ष्यों के लिए लक्ष्य रखते हैं।",
    },
    {
        "en": "That's a stepping stone, a step towards a larger goal.",
        "hi": "यह एक शुरुआती कदम है, एक बड़े लक्ष्य की ओर एक कदम।",
    },
    {
        "en": "It's a whole new ballgame, things have changed significantly.",
        "hi": "यह बिल्कुल नई स्थिति है, चीजें काफी बदल गई हैं।",
    },
    {
        "en": "He's a trendsetter, always ahead of the fashion curve.",
        "hi": "वह नए चलन शुरू करने वाला है, हमेशा फैशन की दौड़ में आगे रहता है।",
    },
    {
        "en": "I'm feeling out of my depth, not qualified for this task.",
        "hi": "मुझे यह थोड़ा मुश्किल लग रहा है, मैं इस कार्य के लिए योग्य नहीं हूँ।",
    },
    {
        "en": "Let's keep it simple, avoid unnecessary complexity.",
        "hi": "चलो इसे सरल रखते हैं, अनावश्यक जटिलता से बचते हैं।",
    },
    {
        "en": "Don't get your hopes up, manage your expectations.",
        "hi": "ज़्यादा उम्मीदें मत रखो, अपनी उम्मीदों को प्रबंधित करो।",
    },
    {
        "en": "He's a diamond in the rough, needs polishing but has potential.",
        "hi": "उसमें प्रतिभा है, पर उसे तराशने की ज़रूरत है, उसे चमकाने की ज़रूरत है लेकिन उसमें क्षमता है।",
    },
    {
        "en": "Let's go with the flow, adapt to changing circumstances.",
        "hi": "चलो जैसे चल रहा है, चलने देते हैं, बदलती परिस्थितियों के अनुकूल होते हैं।",
    },
    {
        "en": "That's a real bummer, a disappointing outcome.",
        "hi": "यह सच में निराशाजनक है, एक निराशाजनक परिणाम।",
    },
    {
        "en": "He's got a chip on his shoulder; always feels slighted.",
        "hi": "वह हमेशा नाराज़ रहता है; हमेशा अपमानित महसूस करता है।",
    },
    {
        "en": "I'm on cloud nine; feeling incredibly happy.",
        "hi": "मैं बहुत खुश हूँ; अविश्वसनीय रूप से खुश महसूस कर रहा हूँ।",
    },
    {
        "en": "Keep your chin up; things will get better.",
        "hi": "हिम्मत मत हारो; चीजें बेहतर होंगी।",
    },
    {
        "en": "Out of the blue; completely unexpected.",
        "hi": "अचानक से; पूरी तरह से अप्रत्याशित।",
    },
    {
        "en": "Rule of thumb; a general principle.",
        "hi": "अनुभव के आधार पर नियम; एक सामान्य सिद्धांत।",
    },
    {
        "en": "That's a tough nut to crack; a difficult problem.",
        "hi": "यह एक मुश्किल समस्या है; एक कठिन समस्या।",
    },
    {
        "en": "Time flies when you're having fun; it passes quickly.",
        "hi": "मज़े करते हुए समय का पता नहीं चलता; यह जल्दी से गुजरता है।",
    },
    {
        "en": "We're in the same boat; facing the same challenges.",
        "hi": "हमारी स्थिति एक जैसी है; एक ही चुनौतियों का सामना करना।",
    },
    {
        "en": "You can't judge a book by its cover; appearances can be deceiving.",
        "hi": "किसी को बाहरी रूप से मत आंकना; दिखावे भ्रामक हो सकते हैं।",
    },
    {
        "en": "He's gone bananas!; acting crazy.",
        "hi": "वह पागल हो गया है!; पागल की तरह अभिनय करना।",
    },
    { "en": "I'm feeling blue; feeling sad.", "hi": "मैं उदास महसूस कर रहा हूँ।" },
    {
        "en": "Let's paint the town red; go out and have fun.",
        "hi": "चलो खूब मस्ती करते हैं; बाहर जाओ और मजे करो।",
    },
    {
        "en": "That's the best thing since sliced bread; a great invention.",
        "hi": "यह तो कमाल की चीज है; एक महान आविष्कार।",
    },
    {
        "en": "To kill two birds with one stone; accomplish two things at once.",
        "hi": "एक तीर से दो शिकार करना; एक साथ दो काम पूरे करना।",
    },
    {
        "en": "Don't count your chickens before they hatch; don't assume success too early.",
        "hi": "सपने सच होने से पहले उन पर भरोसा मत करो; बहुत जल्दी सफलता मान लेना।",
    },
    {
        "en": "Every cloud has a silver lining; there's always something positive.",
        "hi": "हर मुश्किल में कुछ अच्छा छिपा होता है; हमेशा कुछ सकारात्मक होता है।",
    },
    {
        "en": "The early bird catches the worm; being early is advantageous.",
        "hi": "जो जल्दी उठता है, वह पाता है; जल्दी होना फायदेमंद है।",
    },
    {
        "en": "Practice makes perfect; consistent effort leads to improvement.",
        "hi": "अभ्यास से सब संभव है; लगातार प्रयास से सुधार होता है।",
    },
    {
        "en": "Actions speak louder than words; deeds are more important than promises.",
        "hi": "कथनी से करनी बड़ी होती है; कर्म वादों से ज्यादा महत्वपूर्ण हैं।",
    },
    {
        "en": "Where there's a will, there's a way; determination leads to success.",
        "hi": "जहाँ चाह, वहाँ राह; दृढ़ संकल्प सफलता की ओर ले जाता है।",
    },
    { "en": "That's a piece of cake.", "hi": "यह बहुत आसान है।" },
    { "en": "Easy come, easy go.", "hi": "आसानी से आना, आसानी से जाना।" },
    { "en": "Fortune favors the bold.", "hi": "किस्मत बहादुरों का साथ देती है।" },
    { "en": "The apple doesn't fall far from the tree.", "hi": "बेटा बाप जैसा।" },
    {
        "en": "Don't be a salty loser; congratulate them on their success.",
        "hi": "खारे हारे हुए की तरह मत बनो; उनकी सफलता पर उन्हें बधाई दो।",
    },
    {
        "en": "I need to adult today and actually pay my bills.",
        "hi": "मुझे आज बड़ा बनना पड़ेगा और सच में अपने बिल चुकाने पड़ेंगे।",
    },
    {
        "en": "Let's grab some grub; I'm starving.",
        "hi": "चलो कुछ खाना खाते हैं; मुझे बहुत भूख लगी है।",
    },
    {
        "en": "I'm just trying to vibe and enjoy the music.",
        "hi": "मैं बस माहौल में रंगना और संगीत का आनंद लेना चाह रहा हूँ।",
    },
    {
        "en": "That's a hard pass from me; I'm not interested.",
        "hi": "मेरी तरफ से तो बिल्कुल ना है; मुझे कोई दिलचस्पी नहीं है।",
    },
    {
        "en": "He's always flexing his new car; so showy.",
        "hi": "वह हमेशा अपनी नई कार का दिखावा करता रहता है; बहुत दिखावटी है।",
    },
    {
        "en": "That's a major plot twist; I didn't see that coming.",
        "hi": "यह तो कहानी में बहुत बड़ा मोड़ है; मैंने इसकी उम्मीद नहीं की थी।",
    },
    {
        "en": "I'm just trying to take it easy today.",
        "hi": "मैं बस आज आराम करने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "He's such a couch potato; always watching TV.",
        "hi": "वह तो एकदम आलसी है; हमेशा टीवी देखता रहता है।",
    },
    {
        "en": "He's such a techie; always talking about gadgets.",
        "hi": "वह तो एकदम तकनीकी विशेषज्ञ है; हमेशा गैजेट्स के बारे में बात करता रहता है।",
    },
    {
        "en": "I'm just trying to stay positive.",
        "hi": "मैं बस सकारात्मक रहने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "He's such a smooth operator; very charming.",
        "hi": "वह तो बहुत चतुर है; बहुत आकर्षक।",
    },
    {
        "en": "That's a big mood; feeling the same way.",
        "hi": "मेरा भी यही हाल है; वैसा ही महसूस कर रहा हूँ।",
    },
    {
        "en": "He's such a busybody; always interfering in others' affairs.",
        "hi": "वह तो बहुत दखलअंदाज़ है; हमेशा दूसरों के मामलों में टांग अड़ाता है।",
    },
    {
        "en": "He's such a go-getter; very ambitious and proactive.",
        "hi": "वह तो बहुत मेहनती और महत्वाकांक्षी है।",
    },
    {
        "en": "I'm just trying to stay calm.",
        "hi": "मैं बस शांत रहने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "I'm just trying to figure things out.",
        "hi": "मैं बस चीज़ें समझने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "He's such a team player; works well with others.",
        "hi": "वह टीम में मिलकर काम करने वाला है।",
    },
    {
        "en": "That movie was a total tearjerker; I cried so much.",
        "hi": "वह फिल्म पूरी तरह से रुलाने वाली थी; मैं बहुत रोई।",
    },
    {
        "en": "He's a bit of a control freak; he likes everything just so.",
        "hi": "वह थोड़ा नियंत्रण करने वाला है; उसे सब कुछ बिल्कुल वैसा ही पसंद है।",
    },
    {
        "en": "He's a walking encyclopedia; he knows everything.",
        "hi": "वह चलता-फिरता विश्वकोश है; वह सब कुछ जानता है।",
    },
    {
        "en": "Let's touch base later; we can discuss this in more detail.",
        "hi": "चलो बाद में बात करते हैं; हम इस पर और विस्तार से चर्चा कर सकते हैं।",
    },
    {
        "en": "He's a workaholic; he's always working.",
        "hi": "वह काम के प्रति बहुत समर्पित है; वह हमेशा काम करता रहता है।",
    },
    { "en": "Don't beat around the bush.", "hi": "सीधी बात करो।" },
    { "en": "Haste makes waste.", "hi": "जल्दबाजी में काम बिगड़ता है।" },
    {
        "en": "If it ain't broke, don't fix it.",
        "hi": "जो ठीक चल रहा है, उसे मत छेड़ो।",
    },
    { "en": "Laughter is the best medicine.", "hi": "हँसी सबसे अच्छी दवा है।" },
    {
        "en": "Necessity is the mother of invention.",
        "hi": "आवश्यकता आविष्कार की जननी है।",
    },
    {
        "en": "The pen is mightier than the sword.",
        "hi": "कलम तलवार से ज्यादा ताकतवर होती है।",
    },
    {
        "en": "Where there's smoke, there's fire.",
        "hi": "बिना आग के धुआँ नहीं होता।",
    },
    { "en": "You reap what you sow.", "hi": "जैसा करोगे वैसा भरोगे।" },
    {
        "en": "Two wrongs don't make a right.",
        "hi": "गलत काम से सही नतीजा नहीं निकलता।",
    },
    {
        "en": "Actions speak louder than words.",
        "hi": "कथनी से करनी बड़ी होती है।",
    },
    {
        "en": "Birds of a feather flock together.",
        "hi": "एक जैसे लोग साथ रहते हैं।",
    },
    { "en": "He's got a bone to pick with you.", "hi": "उसे तुमसे कुछ शिकायत है।" },
    { "en": "Let's get down to brass tacks.", "hi": "चलो असली मुद्दे पर आते हैं।" },
    { "en": "That's a real kick in the teeth.", "hi": "यह बहुत निराशाजनक है।" },
    { "en": "What are you getting at?", "hi": "तुम क्या कहना चाह रहे हो?" },
    { "en": "You're skating on thin ice.", "hi": "तुम खतरे में हो।" },
    { "en": "Two heads are better than one.", "hi": "एक से भले दो।" },
    { "en": "He's like a fish out of water.", "hi": "वह असहज महसूस कर रहा है।" },
    {
        "en": "Let's not jump to conclusions.",
        "hi": "चलो जल्दबाजी में निष्कर्ष पर नहीं पहुँचते।",
    },
    { "en": "That's a slap in the face.", "hi": "यह अपमानजनक है।" },
    { "en": "What's the fuss about?", "hi": "किस बात का हंगामा है?" },
    {
        "en": "A rolling stone gathers no moss.",
        "hi": "घूमने-फिरने वाले तरक्की नहीं करते।",
    },
    { "en": "Give the devil his due.", "hi": "दुश्मन की भी तारीफ करनी चाहिए।" },
    { "en": "It's a small world.", "hi": "यह दुनिया बहुत छोटी है।" },
    { "en": "Make hay while the sun shines.", "hi": "समय का फायदा उठाओ।" },
    {
        "en": "The devil is in the details.",
        "hi": "बारीकियों पर ध्यान देना ज़रूरी है।",
    },
    {
        "en": "He's playing devil's advocate.",
        "hi": "वह जानबूझकर विपरीत राय रख रहा है।",
    },
    {
        "en": "I'm feeling a bit out of my depth here.",
        "hi": "मुझे यहाँ थोड़ा मुश्किल लग रहा है।",
    },
    {
        "en": "Let's cross that bridge when we come to it.",
        "hi": "जब वह समस्या आएगी तब देखेंगे।",
    },
    { "en": "What's the long and short of it?", "hi": "इसका सार क्या है?" },
    {
        "en": "You're preaching to the choir.",
        "hi": "तुम उसे बता रहे हो जिसे पहले से ही पता है।",
    },
    {
        "en": "A fool and his money are soon parted.",
        "hi": "मूर्ख आदमी जल्दी पैसा खो देता है।",
    },
    { "en": "Blood is thicker than water.", "hi": "अपने तो अपने होते हैं।" },
    {
        "en": "He totally ghosted me after our date; didn't even send a text.",
        "hi": "वह हमारी डेट के बाद पूरी तरह से गायब हो गया; उसने एक मैसेज भी नहीं भेजा।",
    },
    {
        "en": "That party was lit! Everyone was having a blast.",
        "hi": "वह पार्टी एकदम झक्कास थी! सब लोग खूब मजे कर रहे थे।",
    },
    {
        "en": "That's savage! You totally roasted him in that argument.",
        "hi": "यह तो बहुत करारा था! तुमने उस बहस में उसकी पूरी तरह से बखिया उधेड़ दी।",
    },
    {
        "en": "I'm feeling super stoked about the upcoming vacation.",
        "hi": "मैं आने वाली छुट्टी को लेकर बहुत ज़्यादा उत्साहित महसूस कर रहा हूँ।",
    },
    {
        "en": "He's got major swag; always dressed so stylishly.",
        "hi": "उसका तो अलग ही स्वैग है; हमेशा बहुत स्टाइलिश कपड़े पहनता है।",
    },
    {
        "en": "That concert was epic; the band totally killed it.",
        "hi": "वह कॉन्सर्ट शानदार था; बैंड ने तो कमाल कर दिया।",
    },
    {
        "en": "He's such a know-it-all; always correcting people.",
        "hi": "वह तो एकदम पंडित है; हमेशा लोगों को ठीक करता रहता है।",
    },
    {
        "en": "I'm trying to low-key plan a surprise party for her.",
        "hi": "मैं चुपके से उसके लिए एक सरप्राइज पार्टी प्लान करने की कोशिश कर रहा हूँ।",
    },
    {
        "en": "That's sus; I don't trust what he's saying.",
        "hi": "यह संदिग्ध है; मुझे उसकी बातों पर भरोसा नहीं है।",
    },
    {
        "en": "He's a total simp for her; does anything she asks.",
        "hi": "वह तो उसके लिए एकदम लट्टू है; वह जो भी कहती है, वह करता है।",
    },
    {
        "en": "She's got her life together; so organized and successful.",
        "hi": "उसने अपनी ज़िंदगी अच्छे से जमा ली है; बहुत व्यवस्थित और सफल है।",
    },
    {
        "en": "I'm feeling pretty boujee today; treating myself.",
        "hi": "आज मैं थोड़ा अमीर महसूस कर रहा हूँ; खुद को ट्रीट दे रहा हूँ।",
    },
    {
        "en": "He's always mansplaining things; so annoying.",
        "hi": "वह हमेशा मर्दों की तरह समझाता रहता है; बहुत परेशान करने वाला है।",
    },
    {
        "en": "That's a grey area; not clearly defined.",
        "hi": "यह अस्पष्ट क्षेत्र है।",
    },
    {
        "en": "He's got a real silver tongue; he can charm anyone.",
        "hi": "उसके पास सच में चांदी की जुबान है; वह किसी को भी मोहित कर सकता है।",
    },
    {
        "en": "That's a grey area; it's not clearly defined.",
        "hi": "यह अस्पष्ट क्षेत्र है; यह स्पष्ट रूप से परिभाषित नहीं है।",
    },
    {
        "en": "That's a tough break; it's really unfortunate.",
        "hi": "यह बहुत बुरा हुआ; यह सच में दुर्भाग्यपूर्ण है।",
    },
    {
        "en": "After all that drama, she just ghosted him; didn't even have the decency to explain.",
        "hi": "इतने ड्रामे के बाद, वह उसे बिना कुछ बताए गायब हो गई; उसने समझाने की शालीनता तक नहीं दिखाई।",
    },
    {
        "en": "He's living in la-la land if he thinks they'll actually agree to that proposal.",
        "hi": "वह ख्याली दुनिया में जी रहा है अगर उसे लगता है कि वे वास्तव में उस प्रस्ताव को मान लेंगे।",
    },
    {
        "en": "Don't get your knickers in a twist; it's really not that big of a deal.",
        "hi": "इतना परेशान मत हो; यह वास्तव में इतनी बड़ी बात नहीं है।",
    },
    {
        "en": "The whole project went belly up when the funding fell through.",
        "hi": "जब फंडिंग रुक गई तो पूरी परियोजना चौपट हो गई।",
    },
    {
        "en": "She's throwing shade at her ex on social media again; it's getting old.",
        "hi": "वह सोशल मीडिया पर फिर से अपने पूर्व प्रेमी पर ताना मार रही है; यह अब पुराना हो रहा है।",
    },
    {
        "en": "Let's circle back on this after we've had a chance to mull it over.",
        "hi": "चलो इस पर फिर से बात करते हैं जब हमें इस पर विचार करने का मौका मिल जाएगा।",
    },
    {
        "en": "He's been burning the candle at both ends lately; no wonder he's exhausted.",
        "hi": "वह आजकल बहुत ज़्यादा काम कर रहा है; कोई आश्चर्य नहीं कि वह थका हुआ है।",
    },
    {
        "en": "That's a real kick in the teeth; I was really hoping for that promotion.",
        "hi": "यह वास्तव में निराशाजनक है; मैं वास्तव में उस पदोन्नति की उम्मीद कर रहा था।",
    },
    {
        "en": "I had to eat crow after I made that incorrect prediction.",
        "hi": "मुझे वह गलत भविष्यवाणी करने के बाद शर्मिंदा होना पड़ा।",
    },
    {
        "en": "She's playing hard to get, but I think she's actually interested.",
        "hi": "वह भाव खा रही है, लेकिन मुझे लगता है कि वह वास्तव में दिलचस्पी रखती है।",
    },
    {
        "en": "He totally bailed on our plans last minute; I'm so annoyed.",
        "hi": "उसने आखिरी समय पर हमारी योजना रद्द कर दी; मैं बहुत नाराज़ हूँ।",
    },
    {
        "en": "That news really threw me for a loop; I wasn't expecting that at all.",
        "hi": "उस खबर ने मुझे वास्तव में चौंका दिया; मुझे बिल्कुल भी उम्मीद नहीं थी।",
    },
    {
        "en": "She's got a chip on her shoulder about not getting into that college.",
        "hi": "उसे उस कॉलेज में दाखिला नहीं मिलने का बहुत दुख है।",
    },
    {
        "en": "Let's just call it a day; I'm too tired to continue working on this.",
        "hi": "चलो आज के लिए बस इतना ही करते हैं; मैं इस पर काम करने के लिए बहुत थका हुआ हूँ।",
    },
    {
        "en": "He's always trying to cut corners; it's going to catch up with him eventually.",
        "hi": "वह हमेशा शॉर्टकट लेने की कोशिश करता है; यह अंततः उसे महंगा पड़ेगा।",
    },
    {
        "en": "I'm feeling a bit out of sorts today; I think I need some rest.",
        "hi": "मैं आज थोड़ा असहज महसूस कर रहा हूँ; मुझे लगता है कि मुझे कुछ आराम की ज़रूरत है।",
    },
    {
        "en": "She's really hitting her stride in her new role; she's doing great.",
        "hi": "वह अपनी नई भूमिका में वास्तव में अच्छा कर रही है; वह बहुत अच्छा कर रही है।",
    },
    {
        "en": "Don't beat around the bush; just tell me what you really think.",
        "hi": "बात को घुमा फिरा कर मत कहो; बस मुझे बताओ कि तुम वास्तव में क्या सोचते हो।",
    },
    {
        "en": "He's such a drama queen; he always exaggerates everything.",
        "hi": "वह बहुत ड्रामेबाज़ है; वह हमेशा हर बात को बढ़ा-चढ़ाकर बताता है।",
    },
    {
        "en": "I'm so over this; I need a vacation.",
        "hi": "मैं इससे तंग आ गया हूँ; मुझे छुट्टी चाहिए।",
    },
    {
        "en": "A penny for your thoughts.",
        "hi": "आप क्या सोच रहे हैं?",
    },
    {
        "en": "Actions speak louder than words.",
        "hi": "बातें करने से ज़्यादा काम करना मायने रखता है।",
    },
    {
        "en": "Add insult to injury.",
        "hi": "जले पर नमक छिड़कना।",
    },
    {
        "en": "All bark and no bite.",
        "hi": "गरजने वाले बादल बरसते नहीं।",
    },
    {
        "en": "All in good time.",
        "hi": "सब समय की बात है।",
    },
    {
        "en": "An apple a day keeps the doctor away.",
        "hi": "एक सेब हर दिन खाने से डॉक्टर दूर रहता है।",
    },
    {
        "en": "As easy as pie.",
        "hi": "बहुत आसान।",
    },
    {
        "en": "At the drop of a hat.",
        "hi": "तुरंत।",
    },
    {
        "en": "Back to square one.",
        "hi": "फिर से शुरू करना।",
    },
    {
        "en": "Ball is in your court.",
        "hi": "अब तुम्हारी बारी है।",
    },
    {
        "en": "Barking up the wrong tree.",
        "hi": "गलत दिशा में प्रयास करना।",
    },
    {
        "en": "Beat around the bush.",
        "hi": "बात को घुमा फिरा कर कहना।",
    },
    {
        "en": "Best of both worlds.",
        "hi": "दोनों तरफ से फ़ायदा।",
    },
    {
        "en": "Bite off more than you can chew.",
        "hi": "क्षमता से ज़्यादा काम लेना।",
    },
    {
        "en": "Blessing in disguise.",
        "hi": "आपत्ति में भी अवसर।",
    },
    {
        "en": "Burn the midnight oil.",
        "hi": "देर रात तक काम करना।",
    },
    {
        "en": "By the skin of your teeth.",
        "hi": "बाल-बाल बचना।",
    },
    {
        "en": "Call it a day.",
        "hi": "आज के लिए बस।",
    },
    {
        "en": "Call the shots.",
        "hi": "हुक्म चलाना।",
    },
    {
        "en": "Carry a torch for someone.",
        "hi": "किसी से प्यार करना।",
    },
    {
        "en": "Cast pearls before swine.",
        "hi": "मूर्ख के आगे ज्ञान की बातें करना।",
    },
    {
        "en": "Caught between two stools.",
        "hi": "दुविधा में होना।",
    },
    {
        "en": "Chip on your shoulder.",
        "hi": "नाराज़ होना।",
    },
    {
        "en": "Clean as a whistle.",
        "hi": "पूरी तरह से साफ़।",
    },
    {
        "en": "Clear as mud.",
        "hi": "पूरी तरह से अस्पष्ट।",
    },
    {
        "en": "Close but no cigar.",
        "hi": "लगभग सफल, पर पूरी तरह से नहीं।",
    },
    {
        "en": "Cold turkey.",
        "hi": "अचानक से छोड़ देना।",
    },
    {
        "en": "Come hell or high water.",
        "hi": "चाहे कुछ भी हो।",
    },
    {
        "en": "Cost an arm and a leg.",
        "hi": "बहुत महंगा होना।",
    },
    {
        "en": "Cross that bridge when you come to it.",
        "hi": "समय आने पर देखा जाएगा।",
    },
    {
        "en": "Cry over spilt milk.",
        "hi": "व्यर्थ में पछताना।",
    },
    {
        "en": "Cut corners.",
        "hi": "लागत कम करने के लिए कुछ छोड़ देना।",
    },
    {
        "en": "Cut the mustard.",
        "hi": "अपेक्षित स्तर तक अच्छा होना।",
    },
    {
        "en": "Dark horse.",
        "hi": "अप्रत्याशित विजेता।",
    },
    {
        "en": "Dead ringer.",
        "hi": "हमशकल।",
    },
    {
        "en": "Devil's advocate.",
        "hi": "विपक्ष में तर्क देने वाला।",
    },
    {
        "en": "Don't count your chickens before they hatch.",
        "hi": "पहले से उम्मीदें मत लगाओ।",
    },
    {
        "en": "Don't put all your eggs in one basket.",
        "hi": "एक ही चीज़ पर निर्भर मत रहो।",
    },
    {
        "en": "Drastic times call for drastic measures.",
        "hi": "कठिन समय में कठोर कदम उठाने पड़ते हैं।",
    },
    {
        "en": "Draw a blank.",
        "hi": "कुछ याद न आना।",
    },
    {
        "en": "Drive someone up the wall.",
        "hi": "किसी को परेशान करना।",
    },
    {
        "en": "Drop in the bucket.",
        "hi": "बहुत कम मात्रा।",
    },
    {
        "en": "Face the music.",
        "hi": "परिणाम भुगतना।",
    },
    {
        "en": "Fall head over heels.",
        "hi": "प्यार में पड़ना।",
    },
    {
        "en": "Far cry from.",
        "hi": "बहुत अलग।",
    },
    {
        "en": "Few and far between.",
        "hi": "बहुत कम।",
    },
    {
        "en": "Find your feet.",
        "hi": "परिस्थिति के अनुकूल होना।",
    },
    {
        "en": "Fish out of water.",
        "hi": "असहज महसूस करना।",
    },
    {
        "en": "Fit as a fiddle.",
        "hi": "बहुत स्वस्थ।",
    },
    {
        "en": "Food for thought.",
        "hi": "विचार करने योग्य।",
    },
    {
        "en": "For good.",
        "hi": "हमेशा के लिए।",
    },
    {
        "en": "Get a kick out of something.",
        "hi": "किसी चीज़ से आनंद लेना।",
    },
    {
        "en": "Get something off your chest.",
        "hi": "मन की बात कहना।",
    },
    {
        "en": "Get your wires crossed.",
        "hi": "गलत समझना।",
    },
    {
        "en": "Give someone the cold shoulder.",
        "hi": "किसी को अनदेखा करना।",
    },
    {
        "en": "Go back to the drawing board.",
        "hi": "फिर से शुरू करना।",
    },
    {
        "en": "Go for broke.",
        "hi": "सब कुछ दांव पर लगाना।",
    },
    {
        "en": "Goody two-shoes.",
        "hi": "सदाचारी बनने का दिखावा करने वाला।",
    },
    {
        "en": "Grasp at straws.",
        "hi": "निराशा में कुछ भी करने की कोशिश करना।",
    },
    {
        "en": "Have a blast.",
        "hi": "बहुत मज़ा करना।",
    },
    {
        "en": "Have a heart of gold.",
        "hi": "बहुत दयालु होना।",
    },
    {
        "en": "Have a sweet tooth.",
        "hi": "मीठा पसंद होना।",
    },
    {
        "en": "Have eyes in the back of your head.",
        "hi": "सब कुछ देख पाने में सक्षम होना।",
    },
    {
        "en": "Hit the books.",
        "hi": "पढ़ाई करना।",
    },
    {
        "en": "Hit the sack.",
        "hi": "सोने जाना।",
    },
    {
        "en": "In a pickle.",
        "hi": "मुश्किल में होना।",
    },
    {
        "en": "In hot water.",
        "hi": "परेशानी में होना।",
    },
    {
        "en": "It takes two to tango.",
        "hi": "किसी भी काम के लिए दो लोगों की सहमति ज़रूरी है।",
    },
    {
        "en": "Jump on the bandwagon.",
        "hi": "लोकप्रिय चीज़ों का समर्थन करना।",
    },
    {
        "en": "Keep at bay.",
        "hi": "दूर रखना।",
    },
    {
        "en": "Keep your chin up.",
        "hi": "उम्मीद बनाए रखना।",
    },
    {
        "en": "Kill two birds with one stone.",
        "hi": "एक तीर से दो शिकार करना।",
    },
    {
        "en": "Last straw.",
        "hi": "अंतिम परेशानी।",
    },
    {
        "en": "Learn the ropes.",
        "hi": "काम सीखना।",
    },
    {
        "en": "Let sleeping dogs lie.",
        "hi": "पुरानी बातों को मत छेड़ो।",
    },
    {
        "en": "Let the cat out of the bag.",
        "hi": "रहस्य खोलना।",
    },
    {
        "en": "Long shot.",
        "hi": "कम संभावना।",
    },
    {
        "en": "Look before you leap.",
        "hi": "सोच समझकर काम करना।",
    },
    {
        "en": "Lose your touch.",
        "hi": "कौशल खो देना।",
    },
    {
        "en": "Make a mountain out of a molehill.",
        "hi": "बात को बढ़ा-चढ़ाकर कहना।",
    },
    {
        "en": "Miss the boat.",
        "hi": "मौका खो देना।",
    },
    {
        "en": "Neck and neck.",
        "hi": "बहुत करीब की प्रतिस्पर्धा।",
    },
    {
        "en": "No pain, no gain.",
        "hi": "कष्ट सहे बिना कुछ नहीं मिलता।",
    },
    {
        "en": "On cloud nine.",
        "hi": "बहुत खुश।",
    },
    {
        "en": "Once in a blue moon.",
        "hi": "कभी कभार।",
    },
    {
        "en": "Out of the blue.",
        "hi": "अचानक।",
    },
    {
        "en": "Over the moon.",
        "hi": "बहुत खुश।",
    },
    {
        "en": "Paint the town red.",
        "hi": "मौज मस्ती करना।",
    },
    {
        "en": "Pass with flying colors.",
        "hi": "बहुत अच्छे नंबरों से पास होना।",
    },
    {
        "en": "Piece of cake.",
        "hi": "बहुत आसान।",
    },
    {
        "en": "Pull someone's leg.",
        "hi": "किसी की टांग खींचना।",
    },
    {
        "en": "Pull the wool over someone's eyes.",
        "hi": "किसी को धोखा देना।",
    },
    {
        "en": "Put all your eggs in one basket.",
        "hi": "एक ही चीज़ पर निर्भर रहना।",
    },
    {
        "en": "Put your best foot forward.",
        "hi": "सबसे अच्छा प्रयास करना।",
    },
    {
        "en": "Rain or shine.",
        "hi": "चाहे कुछ भी हो।",
    },
    {
        "en": "Raining cats and dogs.",
        "hi": "बहुत तेज़ बारिश होना।",
    },
    {
        "en": "Rock the boat.",
        "hi": "शांति भंग करना।",
    },
    {
        "en": "Rule of thumb.",
        "hi": "अनुभव पर आधारित नियम।",
    },
    {
        "en": "See eye to eye.",
        "hi": "सहमत होना।",
    },
    {
        "en": "See the light.",
        "hi": "समझ में आना।",
    },
    {
        "en": "Shoot for the stars.",
        "hi": "महान लक्ष्य रखना।",
    },
    {
        "en": "Sick and tired.",
        "hi": "तंग आ जाना।",
    },
    {
        "en": "Sit on the fence.",
        "hi": "तटस्थ रहना।",
    },
    {
        "en": "Spill the beans.",
        "hi": "रहस्य खोलना।",
    },
    {
        "en": "Split hairs.",
        "hi": "छोटी बातों पर बहस करना।",
    },
    {
        "en": "Steal someone's thunder.",
        "hi": "किसी और का श्रेय लेना।",
    },
    {
        "en": "Stick to your guns.",
        "hi": "अपने विचारों पर दृढ़ रहना।",
    },
    {
        "en": "Take a chill pill.",
        "hi": "शांत हो जाओ।",
    },
    {
        "en": "Take with a grain of salt.",
        "hi": "पूरी तरह से विश्वास न करना।",
    },
    {
        "en": "Taste of your own medicine.",
        "hi": "अपने कर्मों का फल भुगतना।",
    },
    {
        "en": "The ball is in your court.",
        "hi": "अब तुम्हारी बारी है।",
    },
    {
        "en": "The devil is in the details.",
        "hi": "बारीकियों पर ध्यान देना ज़रूरी है।",
    },
    {
        "en": "The elephant in the room.",
        "hi": "एक बड़ी समस्या जिस पर कोई बात नहीं करना चाहता।",
    },
    {
        "en": "The last straw.",
        "hi": "अंतिम परेशानी।",
    },
    {
        "en": "The more the merrier.",
        "hi": "जितने ज़्यादा, उतना अच्छा।",
    },
    {
        "en": "The pot calling the kettle black.",
        "hi": "अपना दोष दूसरों पर मढ़ना।",
    },
    {
        "en": "Through thick and thin.",
        "hi": "हर परिस्थिति में साथ देना।",
    },
    {
        "en": "Tie the knot.",
        "hi": "शादी करना।",
    },
    {
        "en": "Time flies.",
        "hi": "समय बहुत जल्दी बीत जाता है।",
    },
    {
        "en": "To get bent out of shape.",
        "hi": "परेशान होना।",
    },
    {
        "en": "To hear something straight from the horse's mouth.",
        "hi": "सीधे विश्वसनीय स्रोत से सुनना।",
    },
    {
        "en": "To kill two birds with one stone.",
        "hi": "एक तीर से दो शिकार करना।",
    },
    {
        "en": "To make a long story short.",
        "hi": "संक्षेप में।",
    },
    {
        "en": "To miss the boat.",
        "hi": "मौका खो देना।",
    },
    {
        "en": "To pull someone's leg.",
        "hi": "किसी की टांग खींचना।",
    },
    {
        "en": "To see eye to eye.",
        "hi": "सहमत होना।",
    },
    {
        "en": "To sit on the fence.",
        "hi": "तटस्थ रहना।",
    },
    {
        "en": "Under the weather.",
        "hi": "बीमार महसूस करना।",
    },
    {
        "en": "Up a creek without a paddle.",
        "hi": "मुश्किल में होना।",
    },
    {
        "en": "He ghosted me faster than my salary disappears by the 5th of every month.",
        "hi": "वो ऐसे गायब हुआ जैसे तनख्वाह पांच तारीख को हो जाती है।",
    },
    {
        "en": "She's got more drama than a daily soap running since the '90s.",
        "hi": "उसके अंदर इतना ड्रामा है जितना 90 के दशक के सीरियल्स में होता था।",
    },
    {
        "en": "You can't expect Wi-Fi strength and emotional strength from the same person.",
        "hi": "किसी एक इंसान से वाई-फाई की स्पीड और इमोशनल स्ट्रेंथ दोनों की उम्मीद मत रखो।",
    },
    {
        "en": "My life is like a group project—90% confusion and 10% panic.",
        "hi": "मेरी ज़िंदगी किसी ग्रुप प्रोजेक्ट जैसी है—90% कन्फ्यूजन और 10% पैनिक।",
    },
    {
        "en": "He acts like he's allergic to responsibility—sneezes every time commitment comes up.",
        "hi": "वो ज़िम्मेदारी से ऐसा भागता है जैसे एलर्जी हो—कमिटमेंट का नाम आते ही छींकने लगता है।",
    },
    {
        "en": "Don't worry, her attitude can fit into a thimble—but somehow it still manages to spill everywhere.",
        "hi": "उसका एटीट्यूड तो अंगूठे में समा जाए, लेकिन फिर भी हर जगह फैल जाता है।",
    },
    {
        "en": "He has opinions about everything, even the tea temperature in random strangers' cups.",
        "hi": "उसे हर चीज़ पर राय देनी है, चाहे किसी अजनबी की चाय कितनी गर्म हो उसी पर भी।",
    },
    {
        "en": "I asked for a sign from the universe, and it replied with 'LOL, good luck'.",
        "hi": "मैंने ब्रह्मांड से कोई इशारा मांगा, और जवाब में 'LOL, गुड लक' मिला।",
    },
    {
        "en": "She rolls her eyes so hard, I'm afraid they might get stuck at the back of her head one day.",
        "hi": "वो आंखें ऐसे घुमाती है जैसे एक दिन पीछे ही फंस जाएंगी।",
    },
    {
        "en": "His ambition peaked when he chose the largest fries instead of the medium ones.",
        "hi": "उसकी सबसे बड़ी महत्वाकांक्षा थी मीडियम फ्राईज़ छोड़कर बड़े फ्राईज़ लेना।",
    },
    {
        "en": "No amount of yoga can help you stretch the truth as much as he does.",
        "hi": "इतना झूठ कोई योगा में भी नहीं फैलाता जितना वो फैला देता है।",
    },
    {
        "en": "You bring chaos wherever you go—like a toddler with a permanent marker.",
        "hi": "तू जहां जाता है वहां तूफान मचता है, जैसे बच्चा परमानेंट मार्कर लेकर घूम रहा हो।",
    },
    {
        "en": "Waking up early is my body's way of telling me that adulthood is a scam.",
        "hi": "सुबह जल्दी उठना मेरे शरीर का ये कहने का तरीका है कि अडल्टिंग एक धोखा है।",
    },
    {
        "en": "His sense of fashion is what happens when you let a raccoon loose in a wardrobe.",
        "hi": "उसका फैशन सेंस ऐसा है जैसे अलमारी में कोई रैकून घुस गया हो।",
    },
    {
        "en": "She's not high maintenance, just allergic to average.",
        "hi": "वो हाई मेंटेनेंस नहीं है, बस औसत चीज़ों से एलर्जी है।",
    },
    {
        "en": "Trying to reason with him is like arguing with a YouTube ad—you'll skip it eventually.",
        "hi": "उससे बहस करना ऐसा है जैसे यूट्यूब ऐड से तर्क करना—आखिरकार स्किप करना ही पड़ता है।",
    },
    {
        "en": "My brain at 2 AM: let's rethink every embarrassing thing since 2008.",
        "hi": "रात 2 बजे दिमाग: चलो 2008 से अब तक की सारी शर्मनाक बातें फिर से याद करें।",
    },
    {
        "en": "She replies to texts like they're part of a sacred ritual—one response every full moon.",
        "hi": "वो मैसेज का जवाब ऐसे देती है जैसे कोई पवित्र रिवाज़ हो—हर पूर्णिमा पर एक बार।",
    },
    {
        "en": "He said 'I'll be there in 5 minutes' and I aged 3 years waiting.",
        "hi": "'5 मिनट में आ रहा हूं' कहकर उसने मुझे बूढ़ा कर दिया।",
    },
    {
        "en": "Her sarcasm is so sharp, it needs a license like a weapon.",
        "hi": "उसका व्यंग्य इतना धारदार है कि उसे हथियार लाइसेंस की जरूरत होनी चाहिए।",
    },
    {
        "en": "She cooks like it's a MasterChef finale, but the smoke alarm disagrees.",
        "hi": "वो ऐसे खाना बनाती है जैसे मास्टरशेफ का फिनाले हो, लेकिन स्मोक अलार्म कुछ और ही कहता है।",
    },
    {
        "en": "If laziness was a sport, he'd be the undefeated champion with zero effort.",
        "hi": "अगर आलस्य खेल होता, तो वो बिना मेहनत किए चैंपियन होता।",
    },
    {
        "en": "He flexes his gym selfies more than his actual muscles.",
        "hi": "उसकी जिम की सेल्फियां उसके मसल्स से ज़्यादा स्ट्रॉन्ग हैं।",
    },
    {
        "en": "Her playlist has more break-up songs than her actual break-ups.",
        "hi": "उसकी प्लेलिस्ट में ब्रेकअप सॉन्ग्स की गिनती उसके असली ब्रेकअप से ज़्यादा है।",
    },
    {
        "en": "Don't take his promises seriously, even Google Maps can't track that route.",
        "hi": "उसके वादों को सीरियसली मत लो, गूगल मैप्स भी वो रास्ता नहीं पकड़ सकता।",
    },
    {
        "en": "He dresses like every day is a fashion emergency at 2 AM.",
        "hi": "वो ऐसे कपड़े पहनता है जैसे हर दिन रात 2 बजे की फैशन इमरजेंसी हो।",
    },
    {
        "en": "She multitasks like a desi mom—making chai, gossiping, and giving life advice all at once.",
        "hi": "वो देसी माँ की तरह मल्टीटास्क करती है—चाय बनाना, गॉसिप करना, और लाइफ एडवाइस देना एक साथ।",
    },
    {
        "en": "My luck is like a flat tire—never shows up when you really need it.",
        "hi": "मेरी किस्मत तो पंचर टायर जैसी है—जब ज़रूरत हो तब साथ नहीं देती।",
    },
    {
        "en": "He talks like he's narrating a Netflix docuseries, but with no facts.",
        "hi": "वो ऐसे बोलता है जैसे नेटफ्लिक्स की डॉक्यूसीरी सुना रहा हो, पर बिना फैक्ट्स के।",
    },
    {
        "en": "She's always 'on a break', even when life hasn't hired her yet.",
        "hi": "वो हमेशा 'ब्रेक' पर रहती है, जबकि ज़िंदगी ने अब तक नौकरी दी भी नहीं।",
    },
    {
        "en": "His idea of helping is standing nearby and saying 'you got this'.",
        "hi": "उसकी मदद का तरीका है पास खड़े रहना और कहना 'तू कर लेगा'।",
    },
    {
        "en": "Every time I save money, my online cart gets feelings.",
        "hi": "जैसे ही पैसे बचते हैं, मेरा ऑनलाइन कार्ट भावुक हो जाता है।",
    },
    {
        "en": "She eats salad like she's being punished by a gym trainer's ghost.",
        "hi": "वो सलाद ऐसे खाती है जैसे किसी जिम ट्रेनर के भूत ने डरा रखा हो।",
    },
    {
        "en": "He gives advice like he's on TED Talk, but lives like a reality show contestant.",
        "hi": "वो सलाह ऐसे देता है जैसे TED Talk पर हो, लेकिन ज़िंदगी किसी रियलिटी शो जैसी है।",
    },
    {
        "en": "My sleep schedule is like a broken vending machine—unpredictable and disappointing.",
        "hi": "मेरी नींद की टाइमिंग ऐसी है जैसे कोई टूटी वेंडिंग मशीन—कभी भी चालू नहीं रहती।",
    },
    {
        "en": "She has a PhD in stalking and a master's in overthinking.",
        "hi": "उसके पास स्टॉकिंग में पीएचडी और ओवरथिंकिंग में मास्टर्स है।",
    },
    {
        "en": "He celebrates small wins like he won a national award for finding parking.",
        "hi": "वो छोटे-छोटे जीत ऐसे मनाता है जैसे पार्किंग मिलने पर नेशनल अवॉर्ड मिला हो।",
    },
    {
        "en": "My boss thinks 'urgent' means 'drop everything including your life'.",
        "hi": "मेरे बॉस के लिए 'जरूरी' का मतलब है—सब कुछ छोड़ो, ज़िंदगी भी।",
    },
    {
        "en": "He reads one book and starts quoting it like it's the constitution.",
        "hi": "एक किताब पढ़कर ऐसे कोट करता है जैसे संविधान लिखा हो।",
    },
    {
        "en": "She gave me a look so judgmental, even my Wi-Fi felt the pressure.",
        "hi": "उसकी नज़र इतनी जजमेंटल थी कि मेरा वाई-फाई भी घबरा गया।",
    },
    {
        "en": "His productivity peaks exactly five minutes before the deadline, powered solely by panic.",
        "hi": "उसकी सारी प्रोडक्टिविटी डेडलाइन से पाँच मिनट पहले ही आती है, वो भी घबराहट के दम पर।",
    },
    {
        "en": "Trying to make plans with her is like chasing a mirage in the desert—never happens.",
        "hi": "उसके साथ प्लान बनाना तो जैसे रेगिस्तान में मृगतृष्णा के पीछे भागना है—हक़ीक़त नहीं बनता।",
    },
    {
        "en": "She posts motivational quotes after crying over a cold coffee breakup.",
        "hi": "वो ब्रेकअप के बाद ठंडी कॉफी पीते हुए मोटिवेशनल कोट्स डालती है।",
    },
    {
        "en": "He has more side hustles than actual income sources.",
        "hi": "उसके पास साइड हसल ज़्यादा हैं, असली इनकम कम।",
    },
    {
        "en": "My brain takes a power nap every time someone mentions 'finance planning'.",
        "hi": "जैसे ही कोई 'फाइनेंस प्लानिंग' कहता है, मेरा दिमाग नींद में चला जाता है।",
    },
    {
        "en": "She's so dramatic, even Ekta Kapoor takes notes.",
        "hi": "वो इतनी ड्रामेबाज़ है कि एकता कपूर भी उससे टिप्स ले।",
    },
    {
        "en": "He's so confused in life, even Google Maps refuses to reroute him.",
        "hi": "वो ज़िंदगी में इतना खोया है कि गूगल मैप्स भी उसे रीरूट करने से मना कर दे।",
    },
    {
        "en": "My relatives have a talent—finding my flaws faster than a fingerprint scanner.",
        "hi": "मेरे रिश्तेदारों को मेरी कमियाँ ढूंढने में उंगलियों के निशान स्कैनर से ज़्यादा स्पीड है।",
    },
    {
        "en": "His idea of budgeting is checking his bank balance and crying softly.",
        "hi": "उसका बजट बनाना मतलब बैलेंस चेक करना और चुपचाप रो लेना।",
    },
    {
        "en": "Every time I try to eat healthy, someone offers me samosas like it's fate testing me.",
        "hi": "जैसे ही हेल्दी खाने का सोचता हूँ, कोई समोसे ले आता है—जैसे किस्मत मेरा इम्तिहान ले रही हो।",
    },
    {
        "en": "She's not ignoring you, she's just busy overthinking your last emoji.",
        "hi": "वो तुम्हें इग्नोर नहीं कर रही, बस तुम्हारे आखिरी इमोजी पर ओवरथिंक कर रही है।",
    },
    {
        "en": "I dress according to my mood, which is mostly 'do I have to go out?'",
        "hi": "मैं मूड के हिसाब से कपड़े पहनता हूँ, जो ज़्यादातर होता है—'क्या बाहर जाना ज़रूरी है?'",
    },
    {
        "en": "He gives relationship advice like a YouTuber with no experience and lots of confidence.",
        "hi": "वो रिलेशनशिप की सलाह ऐसे देता है जैसे कोई यूट्यूबर—अनुभव ज़ीरो, कॉन्फिडेंस फुल।",
    },
    {
        "en": "Her life is basically a playlist of overthinking, impulse buying, and existential dread.",
        "hi": "उसकी ज़िंदगी ओवरथिंकिंग, झटपट शॉपिंग और एक्ज़िस्टेंशियल क्राइसेस का प्लेलिस्ट है।",
    },
    {
        "en": "His idea of deep conversation is asking 'what's your favorite filter?'",
        "hi": "उसके लिए गहरी बातें मतलब—'तेरा फेवरेट इंस्टा फिल्टर कौन सा है?'",
    },
    {
        "en": "Whenever I feel like I have my life together, my phone battery is at 1%.",
        "hi": "जब भी लगता है ज़िंदगी पटरी पर है, फोन की बैटरी 1% पर होती है।",
    },
    {
        "en": "She scrolls reels so fast, even Instagram gets dizzy.",
        "hi": "वो इतनी तेजी से रील्स स्क्रॉल करती है कि इंस्टाग्राम को भी चक्कर आ जाए।",
    },
    {
        "en": "He shops like there's an apocalypse coming and discounts are the cure.",
        "hi": "वो ऐसे शॉपिंग करता है जैसे दुनिया खत्म होने वाली है और छूट ही इलाज है।",
    },
    {
        "en": "Every group has that one person who replies in the group chat after 3 days—with 'LOL'.",
        "hi": "हर ग्रुप में एक बंदा होता है जो तीन दिन बाद सिर्फ 'LOL' लिखकर जवाब देता है।",
    },
    {
        "en": "She blocks and unblocks people like it's part of her skincare routine.",
        "hi": "वो लोगों को ऐसे ब्लॉक-अनब्लॉक करती है जैसे स्किनकेयर का हिस्सा हो।",
    },
    {
        "en": "He attends Zoom meetings like he's on a secret undercover mission—camera off, mic off, presence questionable.",
        "hi": "वो ज़ूम मीटिंग ऐसे करता है जैसे कोई सीक्रेट मिशन पर हो—कैमरा बंद, माइक बंद, मौजूदगी संदिग्ध।",
    },
    {
        "en": "Her version of 'I'll be ready in 5 minutes' needs its own time zone.",
        "hi": "'मैं पाँच मिनट में तैयार हूँ' वाली उसकी टाइमिंग के लिए अलग टाइम ज़ोन चाहिए।",
    },
    {
        "en": "I didn't choose the broke life, the broke life chose me—with monthly EMIs.",
        "hi": "मैंने कंगाली नहीं चुनी, कंगाली ने मुझे चुना—हर महीने की EMI के साथ।",
    },
    {
        "en": "He sends voice notes longer than Bollywood songs—no rewind, only regret.",
        "hi": "उसके वॉइस नोट्स बॉलीवुड गानों से भी लंबे होते हैं—ना रिवाइंड, सिर्फ पछतावा।",
    },
    {
        "en": "She's so into astrology, she won't date a guy if Mercury is in retrograde.",
        "hi": "वो एस्ट्रोलॉजी में इतनी डूबी है कि मर्करी रेट्रोग्रेड में डेट भी नहीं करती।",
    },
    {
        "en": "His fashion sense screams 'influencer', but his budget whispers 'end of season sale'.",
        "hi": "उसका फैशन स्टाइल कहता है 'इन्फ्लुएंसर', पर बजट चुपचाप कहता है 'सेल में लिया था'।",
    },
    {
        "en": "I start dieting every Monday and attend a food festival by Wednesday.",
        "hi": "हर सोमवार डाइट शुरू करता हूँ और बुधवार को फूड फेस्टिवल में दिख जाता हूँ।",
    },
    {
        "en": "She writes captions like she's submitting a thesis on heartbreak.",
        "hi": "वो इंस्टा कैप्शन ऐसे लिखती है जैसे ब्रेकअप पर थीसिस जमा कर रही हो।",
    },
    {
        "en": "He thinks being sarcastic is a personality trait—and honestly, it kinda works.",
        "hi": "उसे लगता है कि सर्काज़्म एक पर्सनैलिटी है—और सच कहूँ तो चल भी जाता है।",
    },
    {
        "en": "My bank account and my stomach are always empty at the same time—it's spiritual alignment.",
        "hi": "मेरा बैंक अकाउंट और पेट एक साथ खाली रहते हैं—इसे ही तो आध्यात्मिक संतुलन कहते हैं।",
    },
    {
        "en": "She adds 'haha' at the end of texts like a disclaimer to avoid sounding mean.",
        "hi": "वो मैसेज के अंत में 'हाहा' ऐसे जोड़ती है जैसे कोई डिस्क्लेमर हो—कि बुरा मत मानना।",
    },
    {
        "en": "He treats gym like a long-distance relationship—rare visits and lots of excuses.",
        "hi": "उसके लिए जिम किसी लॉन्ग डिस्टेंस रिलेशनशिप जैसा है—कभी-कभी जाना और बहानों की भरमार।",
    },
    {
        "en": "My weekend plans are usually a toss-up between sleeping and pretending to be productive.",
        "hi": "मेरे वीकेंड प्लान्स होते हैं—नींद लेना या प्रोडक्टिव दिखने का नाटक करना।",
    },
    {
        "en": "Her Instagram bio changes more frequently than her mood swings.",
        "hi": "उसकी इंस्टा बायो उसके मूड स्विंग्स से ज़्यादा जल्दी बदलती है।",
    },
    {
        "en": "He thinks forwarding good morning messages is a love language.",
        "hi": "उसे लगता है 'गुड मॉर्निंग' मैसेज भेजना प्यार जताने का तरीका है।",
    },
    {
        "en": "She has trust issues with people but blind faith in zodiac memes.",
        "hi": "उसे लोगों पर भरोसा नहीं, पर राशिफल मीम्स पर अंधा विश्वास है।",
    },
    {
        "en": "My idea of a social life is replying to memes in the group chat.",
        "hi": "मेरी सोशल लाइफ का मतलब है—ग्रुप चैट में मीम्स पर रिप्लाय करना।",
    },
    {
        "en": "He flirts like a Bollywood villain—loud, outdated, and totally unaware.",
        "hi": "वो फ्लर्ट ऐसे करता है जैसे कोई बॉलीवुड विलेन—तेज, पुराना और पूरी तरह बेखबर।",
    },
    {
        "en": "She plans her Instagram feed more seriously than her career.",
        "hi": "वो इंस्टा फीड की प्लानिंग अपनी करियर प्लानिंग से ज़्यादा सीरियसली लेती है।",
    },
    {
        "en": "My brain goes on airplane mode the moment someone says 'networking opportunity'.",
        "hi": "जैसे ही कोई 'नेटवर्किंग का मौका' कहता है, मेरा दिमाग एयरप्लेन मोड पर चला जाता है।",
    },
    {
        "en": "His idea of romance is sharing the last bite of Maggi without being asked.",
        "hi": "उसके लिए रोमांस मतलब बिना पूछे मैगी का आखिरी कौर शेयर करना।",
    },
    {
        "en": "Every time I try to save money, my phone decides to fall and crack its screen.",
        "hi": "जैसे ही पैसे बचाने की सोचता हूँ, मेरा फोन गिर के स्क्रीन तोड़ देता है।",
    },
    {
        "en": "Her personality is 80% coffee, 10% overthinking, and 10% shopping carts she never checks out.",
        "hi": "उसकी पर्सनैलिटी का 80% हिस्सा कॉफी, 10% ओवरथिंकिंग और 10% शॉपिंग कार्ट्स हैं जो कभी चेकआउट नहीं होतीं।",
    },
    {
        "en": "He claims to be low maintenance but throws tantrums when there's no WiFi.",
        "hi": "वो खुद को लो मेंटेनेंस बोलता है लेकिन WiFi न मिले तो नखरे शुरू।",
    },
    {
        "en": "I didn't ghost you, I just socially hibernated for a few months.",
        "hi": "मैंने तुम्हें इग्नोर नहीं किया, बस सोशल हाइबरनेशन में चला गया था कुछ महीनों के लिए।",
    },
    {
        "en": "Her phone battery survives longer than most of her relationships.",
        "hi": "उसकी फोन बैटरी उसके रिलेशनशिप्स से ज़्यादा टिकाऊ है।",
    },
    {
        "en": "He says he's 'chill', but argues with the Swiggy delivery guy over 5 minutes delay.",
        "hi": "वो खुद को 'चिल' कहता है, लेकिन स्विगी वाले से 5 मिनट की देरी पर झगड़ लेता है।",
    },
    {
        "en": "I meditate, but mostly on whether I should text them or not.",
        "hi": "मैं मेडिटेट करता हूँ, पर ज़्यादातर इस पर कि उसे मैसेज करूं या नहीं।",
    },
    {
        "en": "She gets ready like she's walking the red carpet, even if it's just a grocery run.",
        "hi": "वो ऐसे तैयार होती है जैसे रेड कार्पेट पर जा रही हो, भले ही सब्ज़ी लेने निकली हो।",
    },
    {
        "en": "His WiFi password has more emotional depth than his conversations.",
        "hi": "उसका WiFi पासवर्ड उसकी बातचीत से ज़्यादा इमोशनल डेप्थ रखता है।",
    },
    {
        "en": "Every family function is basically free food with side servings of judgment.",
        "hi": "हर फैमिली फंक्शन फ्री खाना होता है, साथ में ताने फ्री ऑफ़ कॉस्ट मिलते हैं।",
    },
    {
        "en": "She shops like she's prepping for an imaginary vacation she'll never go on.",
        "hi": "वो ऐसे शॉपिंग करती है जैसे किसी काल्पनिक ट्रिप की तैयारी कर रही हो जो कभी होगी ही नहीं।",
    },
    {
        "en": "His flirting is so subtle, even Google Translate can't decode it.",
        "hi": "उसका फ्लर्टिंग इतना सब्टल है कि गूगल ट्रांसलेट भी हार मान जाए।",
    },
    {
        "en": "I'm on a seafood diet—I see food, I eat it, and regret later.",
        "hi": "मैं सी-फूड डाइट पर हूँ—जो दिखा, खा लिया, फिर पछताया।",
    },
    {
        "en": "She'll ignore your texts but post motivational stories about self-love.",
        "hi": "वो तुम्हारा मैसेज इग्नोर कर देगी, पर स्टोरी में सेल्फ-लव वाले कोट्स ज़रूर डालेगी।",
    },
    {
        "en": "His playlist has more mood swings than a teenage drama queen.",
        "hi": "उसकी प्लेलिस्ट में मूड स्विंग्स की भरमार है, जैसे कोई टीनेज ड्रामा क्वीन हो।",
    },
    {
        "en": "Whenever I decide to clean my room, I end up finding childhood photos and spiral for two hours.",
        "hi": "जब भी कमरा साफ़ करने बैठता हूँ, बचपन की तस्वीरें मिलती हैं और दो घंटे तक भावनाओं में डूबा रहता हूँ।",
    },
    {
        "en": "She believes in karma but still stalks her ex's new girlfriend religiously.",
        "hi": "वो कर्मा में विश्वास करती है, पर फिर भी अपने एक्स की नई गर्लफ्रेंड को रोज़ स्टॉक करती है।",
    },
    {
        "en": "He has more unread emails than actual responsibilities.",
        "hi": "उसके पास ज़िम्मेदारियों से ज़्यादा अनरीड ईमेल्स हैं।",
    },
    {
        "en": "My bank account is a constant reminder that adulting is a scam.",
        "hi": "मेरा बैंक अकाउंट हमेशा याद दिलाता है कि अडल्टिंग असल में एक बहुत बड़ा धोखा है।",
    },
    {
        "en": "Every time I plan a solo trip, my relatives treat it like I'm eloping with my dreams.",
        "hi": "जब भी मैं सोलो ट्रिप प्लान करता हूँ, रिश्तेदार ऐसे रिएक्ट करते हैं जैसे मैं सपनों के साथ भाग रहा हूँ।",
    },
    {
        "en": "She's the kind of person who'll cancel plans but still post 'missing the vibe' on Instagram.",
        "hi": "वो प्लान कैंसिल भी कर देगी, लेकिन इंस्टा पर 'missing the vibe' ज़रूर डालेगी।",
    },
    {
        "en": "He believes he's a deep thinker because he stares at the ceiling while listening to Lo-fi beats.",
        "hi": "उसे लगता है कि वो बहुत गहरा सोचने वाला है क्योंकि वो लो-फाई सुनते हुए छत ताकता है।",
    },
    {
        "en": "My sleep schedule is like a broken Indian train—delayed, unpredictable, and always off-track.",
        "hi": "मेरी नींद की टाइमिंग किसी देसी ट्रेन जैसी है—लेट, अनप्लान्ड और पटरी से उतरी हुई।",
    },
    {
        "en": "She switches accents faster than WiFi networks at a café.",
        "hi": "वो कैफ़े में WiFi बदलने से भी तेज़ एक्सेंट बदलती है।",
    },
    {
        "en": "His confidence is built entirely on gym selfies and protein shake sponsorships.",
        "hi": "उसका कॉन्फिडेंस पूरी तरह जिम सेल्फ़ीज़ और प्रोटीन शेक के स्पॉन्सरशिप्स पर टिका है।",
    },
    {
        "en": "Every group chat has that one person who only shows up when there's drama or biryani.",
        "hi": "हर ग्रुप चैट में एक बंदा होता है जो सिर्फ ड्रामा या बिरयानी के टाइम पे एक्टिव होता है।",
    },
    {
        "en": "I told my parents I wanted a break and they handed me a broom and mop.",
        "hi": "मैंने मम्मी-पापा से ब्रेक मांगा, उन्होंने झाड़ू-पोछा थमा दिया।",
    },
    {
        "en": "Her idea of 'me-time' is scrolling through memes and judging everyone silently.",
        "hi": "उसके लिए 'मी-टाइम' का मतलब है मीम्स स्क्रॉल करना और सबको मन ही मन जज करना।",
    },
    {
        "en": "He uses 'I'm an introvert' as an excuse to avoid every family gathering, including his own birthday.",
        "hi": "'मैं इंट्रोवर्ट हूँ' वाला उसका बहाना इतना स्ट्रॉन्ग है कि अपने ही बर्थडे पर फैमिली फंक्शन से बच जाता है।",
    },
]


In [24]:
# 6) Build a Hugging Face Dataset from that list :contentReference[oaicite:1]{index=1}
raw_datasets = Dataset.from_list(colloquial_dataset)

# 7) Split into train/test
# raw_datasets = raw_ds.train_test_split(test_size=0.2, seed=42)
# print(raw_datasets)


In [25]:
# 8) Preprocessing function
def preprocess_function(examples):
    inputs  = ip.preprocess_batch(examples["en"], src_lang="eng_Latn", tgt_lang="hin_Deva")
    targets = ip.preprocess_batch(examples["hi"], src_lang="hin_Deva", tgt_lang="hin_Deva")
    model_inputs = tokenizer(inputs, max_length=256, truncation=True)
    labels = tokenizer(targets, max_length=256, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 9) Tokenize the splits
tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=["en", "hi"]
)


Map:   0%|          | 0/1035 [00:00<?, ? examples/s]

In [31]:
# 10) Data collator & training args
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir="./lora_colloquial",
    per_device_train_batch_size=4 if device=="cuda" else 2,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    learning_rate=3e-4,
    optim="paged_adamw_8bit" if device=="cuda" else "adamw_torch",
    fp16=device=="cuda",
    report_to=["none"],
    save_strategy="no",
    evaluation_strategy="no",
    logging_steps=100,
)


In [32]:
# 11) Trainer & train
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    # eval_dataset=tokenized_datasets["hi"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("Starting fine‑tuning on your colloquial dataset…")
trainer.train()
print("Done!")


Starting fine‑tuning on your colloquial dataset…


Step,Training Loss
100,12.781500
200,12.768800
300,12.765800


Done!


In [40]:
model.save_pretrained("./lora_colloquial")
tokenizer.save_pretrained("./lora_colloquial")


('./lora_colloquial_final/tokenizer_config.json',
 './lora_colloquial_final/special_tokens_map.json',
 './lora_colloquial_final/dict.SRC.json',
 './lora_colloquial_final/dict.TGT.json',
 './lora_colloquial_final/model.SRC',
 './lora_colloquial_final/model.TGT',
 './lora_colloquial_final/added_tokens.json')

In [33]:
!pip install --quiet evaluate sacrebleu bert-score

import numpy as np
import evaluate

# 2) Prepare lists to collect src, pred, ref
srcs_list, preds, refs = [], [], []
batch_size = 16

for i in range(0, len(raw_ds), batch_size):
    batch = raw_ds[i : i + batch_size]
    srcs   = batch["en"]
    targets = batch["hi"]

    # collect sources for COMET
    srcs_list.extend(srcs)

    # preprocess & tokenize
    inputs = ip.preprocess_batch(srcs, src_lang="eng_Latn", tgt_lang="hin_Deva")
    tok = tokenizer(
        inputs,
        padding="longest",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    ).to(device)

    # generate
    with torch.inference_mode():
        outs = model.generate(
            **tok,
            num_beams=5,
            length_penalty=1.0,
            repetition_penalty=1.2,
            max_new_tokens=2048,
            early_stopping=True
        )

    # decode & postprocess
    dec = tokenizer.batch_decode(outs, skip_special_tokens=True)
    dec = ip.postprocess_batch(dec, lang="hin_Deva")

    preds.extend(dec)
    refs.extend(targets)

In [35]:
!pip install -q unbabel-comet

# 3) Sentence‑level metrics
bleu  = evaluate.load("sacrebleu")  # BLEU :contentReference[oaicite:1]{index=1}
chrf  = evaluate.load("chrf")       # chrF
berts = evaluate.load("bertscore")  # BERTScore

bleu_res  = bleu.compute(predictions=preds, references=[[r] for r in refs])
chrf_res  = chrf.compute(predictions=preds, references=[[r] for r in refs])
berts_res = berts.compute(predictions=preds, references=refs, lang="hi")

print(f"BLEU:          {bleu_res['score']:.2f}")
print(f"chrF:          {chrf_res['score']:.2f}")
print(f"BERTScore F1:  {np.mean(berts_res['f1']):.4f}")


BLEU:          28.38
chrF:          45.25
BERTScore F1:  0.8228


In [ ]:
# # 4) DocCOMET (document‑level COMET‑DA)
# #    This takes src, pred, ref triplets and returns one score per example
# comet = evaluate.load("comet")  
# comet_res = comet.compute(
#     sources=srcs_list,
#     predictions=preds,
#     references=refs
# )
# # 'scores' is a list of floats in [0,1]
# print(f"DocCOMET (DA): {np.mean(comet_res['scores']):.4f}")  # COMET‑DA :contentReference[oaicite:2]{index=2}

# # 5) “Embarrassingly Easy” doc‑level BERTScore
# #    Concatenate the previous reference as context :contentReference[oaicite:3]{index=3}
# preds_ctx, refs_ctx = [], []
# for idx in range(len(preds)):
#     prev = refs[idx-1] if idx>0 else ""
#     preds_ctx.append(prev + " " + preds[idx])
#     refs_ctx.append(prev + " " + refs[idx])

# berts_ctx = evaluate.load("bertscore")
# berts_ctx_res = berts_ctx.compute(predictions=preds_ctx, references=refs_ctx, lang="hi")
# print(f"Doc‑level BERTScore F1: {np.mean(berts_ctx_res['f1']):.4f}")

In [15]:
text = """As I wandered through the bustling city streets on a warm, sunny afternoon, 
    the world around me seemed to hum with life. The vibrant colors of flowers blooming 
    in the meticulously maintained parks painted a vivid contrast against the steel and 
    glass of the towering skyscrapers that loomed above, reflecting the bright blue sky 
    in their countless windows. The sunlight, filtering through the canopy of leaves in 
    the small, scattered patches of greenery that dotted the city, cast dappled shadows 
    on the pavement below, where the rhythmic clatter of footsteps and the distant hum 
    of traffic merged into a soothing, urban symphony. Street performers, positioned at 
    nearly every corner, played their guitars, violins, and saxophones, their melodies 
    blending into a medley of sounds that flowed like a river through the streets, carried
    by the laughter and chatter of people spilling out of bustling cafés and restaurants, 
    their faces alight with the joy of an afternoon well spent. Children raced through the 
    playgrounds, their laughter carried on the gentle breeze that occasionally swept 
    through the streets, bringing with it the scent of freshly baked bread from a nearby 
    bakery and the tang of street food sizzling on grills as vendors called out to passersby. 
    In the midst of all this vibrant activity, I found myself swept up in the energy of the 
    city, its pulse echoing in my ears as I walked along, watching the ever-changing tapestry
    of life unfold before me. The city was a living, breathing organism, constantly in motion, 
    and I couldn’t help but marvel at how different this world felt from the one I had known 
    for so long. It hadn’t been that long ago, only a few months, that I had lived in the 
    countryside, surrounded by rolling hills and quiet fields that stretched as far as the 
    eye could see. The mornings there had always been peaceful, almost meditative in their 
    stillness, with only the soft rustle of leaves in the trees, the gentle whisper of the 
    wind as it passed through the tall grasses, and the occasional chirping of birds breaking 
    the silence. I would wake early, just as the first rays of sunlight crept over the horizon, 
    casting a soft, golden light over the landscape. The air would be cool and crisp, filled 
    with the earthy scent of dew-soaked soil and the faint aroma of wildflowers. Those mornings 
    had their own kind of beauty, a beauty that came from the quiet simplicity of nature, from 
    the feeling of being alone in the world, surrounded by nothing but the sounds of the earth 
    waking up. I would sit on the porch of my small cottage, with a cup of steaming coffee in 
    hand, and watch as the mist slowly lifted from the fields, revealing the distant outline 
    of the forest, where the trees stood tall and silent, like sentinels guarding the secrets 
    of the land. Sometimes, if I was lucky, I’d catch a glimpse of a deer or two grazing in the 
    distance, their sleek bodies moving gracefully through the tall grass, unaware of my presence. 
    But despite the peace and tranquility of the countryside, there had always been a part of me 
    that longed for something more, something beyond the quiet, predictable rhythm of rural life. 
    I had spent years surrounded by nature’s beauty, and while I had loved it, I had also begun 
    to feel a sense of restlessness, a yearning for the energy and excitement that only a city 
    could provide. So, when the opportunity came to move to the city, I had taken it without 
    hesitation, packing up my life into a few boxes and leaving behind the familiar comforts of 
    the countryside for the unknown adventures that awaited me in the bustling streets of urban 
    life. Now, as I walked through those very streets, I realized just how much my life had changed 
    in such a short time. The cit y was everything I had hoped it would be, and more. Every day 
    brought something new, something unexpected. One day, I might stumble upon a hidden café tucked 
    away in a narrow alley, its walls covered in ivy, serving the best espresso I had ever tasted. 
    The next, I might find myself standing in the middle of a street festival, surrounded by food 
    stalls offering dishes from every corner of the world, the air filled with the scent of spices 
    and grilled meats, while performers danced and played music in the center of the crowd, their 
    movements a celebration of culture and tradition. The people, too, were different. In the 
    countryside, I had known everyone—every face was familiar, every story already told. But here, 
    in the city, every person I passed was a mystery, a new story waiting to be discovered. There 
    were the artists, sitting in cafés sketching scenes from their imagination; the businesspeople, 
    hurrying to meetings with phones pressed to their ears, their faces a mask of determination; 
    the students, gathered in groups, discussing everything from philosophy to the latest fashion 
    trends; and the tourists, cameras slung around their necks, eyes wide with wonder as they took 
    in the sights of the city. Each of them was on their own journey, their paths crossing with 
    mine for just a brief moment before they continued on their way, leaving behind only the faintest 
    trace of their presence. Yet, despite all the excitement and energy of the city, there were 
    still moments when I found myself missing the quiet solitude of the countryside. There were 
    times, late at night, when the city had finally quieted down, and I would lie awake in my small 
    apartment, listening to the distant sounds of cars passing by on the streets below, that I 
    would think back to those early mornings in the countryside, to the feeling of peace that came 
    from being alone with nature. I would remember the way the first light of dawn had turned the 
    sky a soft shade of pink, the way the wind had whispered through the trees, and the way the 
    world had felt so still, as if it were holding its breath, waiting for the day to begin. But 
    then the sun would rise, casting its golden light over the city once again, and I would step 
    outside, greeted by the sights and sounds of life unfolding around me, and I would remember 
    why I had chosen to leave the quiet behind. The city, with all its noise, its chaos, and its 
    constant movement, had a magic of its own, a magic that drew me in and made me feel alive in 
    a way that the countryside never could. It was a place where anything seemed possible, where 
    every corner held the promise of something new, something unexpected. And as I continued to 
    walk through the streets that day, I knew that, despite the occasional longing for the peace 
    of the countryside, I had found a new home in the heart of the city, where the pulse of life 
    beat strong and steady, carrying me along with it."""

In [ ]:
# text = "He's a night owl."

inputs = ip.preprocess_batch([text], src_lang="eng_Latn", tgt_lang="hin_Deva")
batch  = tokenizer(
    inputs,
    padding="longest",
    truncation=True,
    max_length=2048,
    return_tensors="pt"
).to(device)

# Generate translation
with torch.inference_mode():
    outputs = model.generate(
        **batch,
        num_beams=5,
        length_penalty=1.0,
        repetition_penalty=1.2,
        max_new_tokens=2048,
        early_stopping=True
    )


In [ ]:
# Decode & postprocess
decoded = tokenizer.batch_decode(
    outputs,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True
)
translation = ip.postprocess_batch(decoded, lang="hin_Deva")[0]

print("→", translation)

In [ ]:
'''जैसे ही मैं एक गर्म, धूप भरी दोपहर में शहर की व्यस्त सड़कों पर घूम रहा था, मेरे चारों ओर की दुनिया जीवन से गूंज रही थी। सावधानीपूर्वक बनाए गए पार्कों में खिलते फूलों के जीवंत रंग उन ऊँचे गगनचुंबी इमारतों के स्टील और कांच के खिलाफ एक जीवंत विरोधाभास को चित्रित करते थे जो ऊपर झुके हुए थे, उनकी अनगिनत खिड़कियों में चमकीले नीले आकाश को प्रतिबिंबित करते थे। सूरज की रोशनी, छोटे, बिखरे हुए हरियाली के टुकड़ों में पत्तियों की छतरी से निकलती हुई, नीचे फुटपाथ पर छायाएँ डालती हुई, जहां कदमों की लयबद्ध गड़गड़ाहट और दूर की ट्रैफिक की गूंज एक शांत, शहरी सिम्फनी में मिल जाती थी। लगभग हर कोने पर स्थित, सड़क कलाकार अपने गिटार, वायलिन और सैक्सोफोन बजाते थे, उनकी धुनें सड़कों से नदी की तरह बहने वाली ध्वनियों के मिश्रण में मिल जाती थीं, जो भीड़भाड़ वाले कैफे और रेस्तरां से बाहर निकलने वाले लोगों की हंसी और गपशप से भरी होती थीं, उनके चेहरे एक अच्छी तरह से बिताई गई दोपहर की खुशी से चमकते थे। बच्चे खेल के मैदानों में दौड़ते थे, उनकी हंसी उस हल्की हवा पर चलती थी जो कभी-कभी सड़कों से बहती थी, अपने साथ पास की बेकरी से ताजा बेक की गई रोटी की खुशबू और विक्रेताओं द्वारा राहगीरों को बुलाए जाने पर ग्रिल पर सिज़लिंग स्ट्रीट फूड की टांग लाती थी। इस सारी जीवंत गतिविधि के बीच, मैं खुद को शहर की ऊर्जा में डूबा हुआ पाया, इसकी नाड़ी मेरे कानों में गूंज रही थी जब मैं आगे बढ़ रहा था, मेरे सामने जीवन की हमेशा बदलती हुई टेपेस्ट्री को देखते हुए। शहर एक जीवित, सांस लेने वाला जीव था, लगातार गति में होता था, और मैं इस बात पर आश्चर्यचकित नहीं हो सकता था कि यह दुनिया उस दुनिया से कितनी अलग महसूस होती है जिसे मैं इतने लंबे समय से जानता था। यह बहुत पहले नहीं हुआ था, केवल कुछ महीने पहले, जब मैं ग्रामीण इलाकों में रहता था, लुढ़कती पहाड़ियों और शांत खेतों से घिरा हुआ था जो आंखों तक फैल जाते थे। वहां की सुबह हमेशा शांतिपूर्ण रही थी, अपनी स्थिरता में लगभग ध्यान केंद्रित करती थी, पेड़ों में पत्तियों की नरम गड़गड़ाहट के साथ, हवा की हल्की फुसफुसाहट के साथ जब वह ऊंची घासों से गुजरती थी, और कभी-कभी पक्षियों की चहचहाहट जो चुप्पी तोड़ती थी। मैं जल्दी जागता था, जैसे ही सूरज की रोशनी की पहली किरणें क्षितिज पर चढ़ती थीं, परिदृश्य पर एक नरम, सुनहरा प्रकाश डालती थीं। हवा ठंडी और कुरकुरा होती, ओस से भीगी हुई मिट्टी की मिट्टी की सुगंध और जंगली फूलों की हल्की सुगंध से भरी होती थी। उन सुबहों की अपनी तरह की सुंदरता होती थी, एक सुंदरता जो प्रकृति की शांत सादगी से आती थी, दुनिया में अकेले होने की भावना से, पृथ्वी की आवाज़ों से घिरा हुआ। मैं अपने छोटे से कॉटेज के बरामदे पर बैठता था, हाथ में भाप भरी कॉफी का कप लेकर, और देखता था कि धुंध धीरे-धीरे खेतों से उठती है, जंगल की दूर की रूपरेखा को प्रकट करती है, जहां पेड़ ऊंचे और शांत खड़े होते हैं, जैसे जमीन के रहस्यों की रक्षा करने वाले प्रहरी। कभी-कभी, अगर मैं भाग्यशाली होता, तो मैं दूर एक या दो हिरणों की एक झलक पाता, उनके चिकने शरीर लंबी घास में शालीनता से चलते हुए, मेरी उपस्थिति से अनजान। लेकिन ग्रामीण इलाकों की शांति और शांति के बावजूद, मेरा हमेशा एक हिस्सा रहा है जो कुछ और चाहता था, ग्रामीण जीवन की शांत, पूर्वानुमेय लय से परे कुछ। मैंने प्रकृति की सुंदरता से घिरे वर्षों बिताए थे, और जबकि मुझे यह पसंद था, मैंने भी बेचैनी की भावना महसूस करना शुरू कर दिया था, उस ऊर्जा और उत्साह की लालसा जो केवल एक शहर ही प्रदान कर सकता था। इसलिए, जब शहर में जाने का अवसर आया, तो मैंने इसे बिना किसी हिचकिचाहट के लिया था, अपने जीवन को कुछ डिब्बों में पैक किया था और ग्रामीण इलाकों की परिचित सुविधाओं को उन अज्ञात रोमांचों के लिए पीछे छोड़ दिया था जो शहरी जीवन की व्यस्त सड़कों में मेरा इंतजार कर रहे थे। अब, जब मैं उन सड़कों से गुजर रहा था, तो मुझे एहसास हुआ कि इतनी कम समय में मेरा जीवन कितना बदल गया था। सिट वाई सब कुछ था जिसकी मैंने उम्मीद की थी, और भी अधिक। हर दिन कुछ नया, कुछ अप्रत्याशित लाया था। एक दिन, मैं एक संकीर्ण गली में छिपे हुए एक छिपे हुए कैफे पर ठोकर खा सकता था, उसकी दीवारें आइवी से ढकी हुई थीं, जो मेरे द्वारा चखे गए सबसे अच्छे एस्प्रेसो की सेवा कर रही थीं। अगले दिन, मैं खुद को एक सड़क उत्सव के बीच में खड़ा पा सकता था, दुनिया के हर कोने से व्यंजन पेश करने वाले खाद्य स्टालों से घिरा हुआ, मसालों और ग्रिल्ड मांस की सुगंध से भरी हवा, जबकि कलाकार भीड़ के केंद्र में नाचते और संगीत बजाते थे, उनकी गतिविधियाँ संस्कृति और परंपरा का उत्सव थीं। लोग भी अलग थे। ग्रामीण इलाकों में, मैंने सभी को जाना था-हर चेहरा परिचित था, हर कहानी पहले से ही बताई गई थी। लेकिन यहाँ, शहर में, हर व्यक्ति जो मैं गुजरता था, एक रहस्य था, एक नई कहानी जो खोजे जाने की प्रतीक्षा कर रहा था। वहाँ कलाकार थे, कैफे में बैठकर अपनी कल्पना से दृश्य स्केच कर रहे थे; व्यवसायी, अपने कानों पर दबाए गए फोन के साथ बैठकों में तेजी से जा रहे थे, उनके चेहरे दृढ़ संकल्प का मुखौटा थे; छात्र, समूहों में इकट्ठा हुए, दर्शन से लेकर नवीनतम फैशन रुझानों तक सब कुछ चर्चा कर रहे थे; और पर्यटक, कैमरे अपने गले में लटक रहे थे, आश्चर्य से आंखें चौड़ी कर रहे थे जब वे शहर के दृश्यों को देख रहे थे। उनमें से प्रत्येक अपनी यात्रा पर था, उनके रास्ते बस एक संक्षिप्त क्षण के लिए मेरे साथ पार कर रहे थे इससे पहले कि वे अपने रास्ते पर चलते रहे, केवल अपनी उपस्थिति का सबसे हल्का निशान छोड़ते हुए। फिर भी, शहर की सभी उत्तेजना और ऊर्जा के बावजूद, अभी भी ऐसे क्षण थे जब मैं खुद को ग्रामीण इलाकों के शांत एकांत को याद करता था। कई बार, देर रात, जब शहर अंततः शांत हो गया था, और मैं अपने छोटे से अपार्टमेंट में जागता हुआ लेट जाता, नीचे की सड़कों पर गुजरने वाली कारों की दूर की आवाजें सुनता, कि मैं उन प्रारंभिक सुबहों को सोचूंगा, जो प्रकृति के साथ अकेले रहने से आती थी। मुझे याद होगा कि कैसे सुबह की पहली रोशनी ने आकाश को गुलाबी की नरम छाया बना दिया था, जिस तरह से हवा ने पेड़ों के माध्यम से फुसफुसाया था, और जिस तरह से दुनिया ने इतना शांत महसूस किया था, जैसे कि वह अपना सांस रोक रही थी, दिन शुरू होने का इंतजार कर रही थी। लेकिन फिर सूरज उठेगा, शहर पर एक बार फिर से अपनी सुनहरी रोशनी डालेगा, और मैं बाहर कदम रखूंगा, अपने चारों ओर खुलते जीवन के दृश्यों और ध्वनियों से स्वागत करूंगा, और मुझे याद होगा कि मैंने चुपचाप पीछे क्यों रहना चुना था। शहर, अपने सभी शोर, अपनी अराजकता, और अपने निरंतर आंदोलन के बावजूद, एक अपना जादू था जिसने मुझे आकर्षित किया और मुझे इस तरह से जीवित महसूस कराया कि ग्रामीण क्षेत्र कभी नहीं कर सकता था। यह एक ऐसी जगह थी जहाँ कुछ भी संभव लग रहा था, जहां हर कोना कुछ नया, कुछ अप्रत्याशित का वादा रखता था। और जैसे-जैसे मैं उस दिन सड़कों पर चलता रहा, मुझे पता था कि, ग्रामीण इलाकों की शांति की कभी-कभार लालसा के बावजूद, मुझे शहर के दिल में एक नया घर मिल गया था, जहां जीवन की धड़कन मजबूत और स्थिर धड़कती थी, मुझे इसके साथ ले जाती थी।'''